# Ordering by metrics and retraining phase

## Dataset: Intel

## Experiment configuration 1


## Configuration 2
	2. Incremental guided retraining starting from the original model using the new adversarial inputs and original training set.

In [ ]:
pip install --user tensorflow==2.5

In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model


In [2]:
tf.__version__ == '2.5.0' # Version of tensorflow

True

In [3]:
pwd

'C:\\Users\\fjdur\\Documents\\upc-july\\notebooks'

In [4]:
cd C:/Users/fjdur/Documents/utils//

C:\Users\fjdur\Documents\utils


In [5]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 54D3-6FC8

 Directory of C:\Users\fjdur\Documents\utils

24/07/2022  22:57    <DIR>          .
24/07/2022  22:57    <DIR>          ..
13/08/2022  18:03    <DIR>          __pycache__
24/07/2022  22:57               432 adversarial_creation.py
07/11/2021  18:37            11.457 project_utils.py
17/07/2022  15:27    <DIR>          tmp
13/08/2022  18:01            31.616 utils_guided_retraining.py
               3 File(s)         43.505 bytes
               4 Dir(s)  22.655.545.344 bytes free


In [6]:
# utils for project
#import utils_guided_retraining2 as utils
import utils_guided_retraining as utils

2.5.0
keras
2.5.0


In [7]:
#cd '../notebooks/'

In [8]:
dataset = "mnist"

In [9]:
x_train,y_train = utils.get_data(dataset,"Train",verbose=True)
x_val,y_val = utils.get_data(dataset,"Val",verbose=True)
x_test,y_test = utils.get_data(dataset,"Test",verbose=True)

D:/data_mnist/x_train.npy
x_set len:  48999
D:/data_mnist/y_train.npy
y_set len:  48999
D:/data_mnist/x_val.npy
x_set len:  14000
D:/data_mnist/y_val.npy
y_set len:  14000
D:/data_mnist/x_test.npy
x_set len:  7001
D:/data_mnist/y_test.npy
y_set len:  7001


In [10]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary",verbose=True)

D:/data_adversarial_july/mnist/train_and_adversary.npy
x_set len:  55998
D:/data_adversarial_july/mnist/train_and_adversary_labels.npy
y_set len:  55998


In [11]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6999
6999


In [12]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,'Test_adversarial') 

In [13]:
x_adversary_test = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test))
y_adversary_test = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test))

6999
6999


## ----

In [14]:
# Original model 

model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_02"


model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/models2"
elif(dataset == 'cifar'):
    model_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/models/model_02"

In [15]:
print(model_dir)
model_original = utils.My_model(dataset,True, model_dir)

C:/Users/fjdur/Documents/upc-july/models/models2
Model loaded correctly


In [16]:
print(len(x_adversary_training))

6999


In [17]:
print(x_adversary_training.shape)
len(x_adversary_training)//20

(6999, 28, 28, 1)


349

In [18]:
n_data_points = 340

In [19]:
import gc

## Loading LSA and DSA values 

In [20]:
lsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"


In [21]:
#lsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/lsa_values.npy"


In [22]:
lsa_values = np.load(lsa_direction)[len(x_train):] 
print(len(lsa_values))


6999


In [23]:

# Obtaining top n images by LSA values
top_images_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),x_adversary_training)
top_labels_by_lsa = utils.get_x_of_indexes(list(np.flip(np.argsort(lsa_values))),y_adversary_training)


In [24]:
m = n_data_points
n = 0
image_sets_lsa = []
label_sets_lsa = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_lsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_lsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_lsa)%m))
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m+(len(top_images_by_lsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_lsa_n = np.array(top_images_by_lsa[:n+m])
        top_labels_by_lsa_n = np.array(top_labels_by_lsa[:n+m])
    image_sets_lsa.append(top_images_by_lsa_n)
    label_sets_lsa.append(top_labels_by_lsa_n)
    print(len(top_images_by_lsa_n))
    n += m

0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [27]:
len(image_sets_lsa[-2])

6460

## Training guided by LSA

In [25]:
len(label_sets_lsa)

20

In [26]:
print(model_dir)

C:/Users/fjdur/Documents/upc-july/models/models2


In [27]:
print(model_dir)

models_lsa = []
for i in range(len(label_sets_lsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_lsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/models2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [31]:
models_lsa[0].evaluate(x_test,y_test)

219/219 [==============================] - 26s 10ms/step - loss: 0.0340 - accuracy: 0.9908


[0.036614201962947845, 0.9894300699234009]

In [32]:
n = 0
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n=n+1

Epoch 1/10
6/6 [==============================] - 11s 899ms/step - loss: 0.9379 - accuracy: 0.7676 - val_loss: 0.0874 - val_accuracy: 0.9836
Epoch 2/10
6/6 [==============================] - 5s 909ms/step - loss: 0.5987 - accuracy: 0.8441 - val_loss: 0.1194 - val_accuracy: 0.9766
Epoch 3/10
6/6 [==============================] - 5s 911ms/step - loss: 0.5679 - accuracy: 0.8588 - val_loss: 0.0696 - val_accuracy: 0.9844
Epoch 4/10
6/6 [==============================] - 5s 934ms/step - loss: 0.5325 - accuracy: 0.8735 - val_loss: 0.0680 - val_accuracy: 0.9830
Epoch 5/10
6/6 [==============================] - 5s 965ms/step - loss: 0.3450 - accuracy: 0.8882 - val_loss: 0.0689 - val_accuracy: 0.9837
Epoch 6/10
6/6 [==============================] - 5s 958ms/step - loss: 0.2883 - accuracy: 0.9118 - val_loss: 0.0670 - val_accuracy: 0.9829
Epoch 7/10
6/6 [==============================] - 4s 847ms/step - loss: 0.3828 - accuracy: 0.8735 - val_loss: 0.1105 - val_accuracy: 0.9754
Epoch 8/10
6/6 [===

In [33]:
n=1
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val,epochs=10,batch_size = 64)
n = n+1

Epoch 1/10
11/11 [==============================] - 7s 543ms/step - loss: 0.8291 - accuracy: 0.8459 - val_loss: 0.0506 - val_accuracy: 0.9868
Epoch 2/10
11/11 [==============================] - 5s 514ms/step - loss: 0.4789 - accuracy: 0.8715 - val_loss: 0.0514 - val_accuracy: 0.9882
Epoch 3/10
11/11 [==============================] - 6s 541ms/step - loss: 0.3480 - accuracy: 0.9147 - val_loss: 0.0567 - val_accuracy: 0.9849
Epoch 4/10
11/11 [==============================] - 5s 493ms/step - loss: 0.3074 - accuracy: 0.9209 - val_loss: 0.0784 - val_accuracy: 0.9815
Epoch 5/10
11/11 [==============================] - 5s 515ms/step - loss: 0.1980 - accuracy: 0.9458 - val_loss: 0.0652 - val_accuracy: 0.9814
Epoch 6/10
11/11 [==============================] - 5s 522ms/step - loss: 0.2425 - accuracy: 0.9258 - val_loss: 0.0809 - val_accuracy: 0.9794
Epoch 7/10
11/11 [==============================] - 5s 518ms/step - loss: 0.2479 - accuracy: 0.9296 - val_loss: 0.0601 - val_accuracy: 0.9848
Epoch 

In [34]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 7s 395ms/step - loss: 0.6595 - accuracy: 0.8665 - val_loss: 0.0409 - val_accuracy: 0.9888
Epoch 2/10
16/16 [==============================] - 6s 374ms/step - loss: 0.2928 - accuracy: 0.9277 - val_loss: 0.0564 - val_accuracy: 0.9860
Epoch 3/10
16/16 [==============================] - 6s 374ms/step - loss: 0.3084 - accuracy: 0.9222 - val_loss: 0.0601 - val_accuracy: 0.9852
Epoch 4/10
16/16 [==============================] - 6s 373ms/step - loss: 0.2631 - accuracy: 0.9377 - val_loss: 0.0470 - val_accuracy: 0.9880
Epoch 5/10
16/16 [==============================] - 6s 374ms/step - loss: 0.2616 - accuracy: 0.9388 - val_loss: 0.0498 - val_accuracy: 0.9864
Epoch 6/10
16/16 [==============================] - 6s 376ms/step - loss: 0.2060 - accuracy: 0.9490 - val_loss: 0.0923 - val_accuracy: 0.9792
Epoch 7/10
16/16 [==============================] - 6s 375ms/step - loss: 0.2134 - accuracy: 0.9396 - val_loss: 0.0606 - val_accuracy: 0.9840
Epoc

In [35]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 8s 310ms/step - loss: 0.4896 - accuracy: 0.9112 - val_loss: 0.0960 - val_accuracy: 0.9797
Epoch 2/10
22/22 [==============================] - 6s 292ms/step - loss: 0.3042 - accuracy: 0.9227 - val_loss: 0.0575 - val_accuracy: 0.9856
Epoch 3/10
22/22 [==============================] - 6s 291ms/step - loss: 0.2402 - accuracy: 0.9432 - val_loss: 0.0603 - val_accuracy: 0.9849
Epoch 4/10
22/22 [==============================] - 6s 294ms/step - loss: 0.1878 - accuracy: 0.9548 - val_loss: 0.0844 - val_accuracy: 0.9797
Epoch 5/10
22/22 [==============================] - 6s 294ms/step - loss: 0.2264 - accuracy: 0.9396 - val_loss: 0.0675 - val_accuracy: 0.9841
Epoch 6/10
22/22 [==============================] - 6s 289ms/step - loss: 0.2096 - accuracy: 0.9365 - val_loss: 0.0851 - val_accuracy: 0.9829
Epoch 7/10
22/22 [==============================] - 6s 292ms/step - loss: 0.2074 - accuracy: 0.9549 - val_loss: 0.0646 - val_accuracy: 0.9856
Epoc

In [36]:
print(n) #4
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 9s 279ms/step - loss: 0.4328 - accuracy: 0.9190 - val_loss: 0.0582 - val_accuracy: 0.9844
Epoch 2/10
27/27 [==============================] - 7s 254ms/step - loss: 0.2297 - accuracy: 0.9418 - val_loss: 0.0836 - val_accuracy: 0.9826
Epoch 3/10
27/27 [==============================] - 7s 257ms/step - loss: 0.2077 - accuracy: 0.9504 - val_loss: 0.0548 - val_accuracy: 0.9860
Epoch 4/10
27/27 [==============================] - 7s 268ms/step - loss: 0.2091 - accuracy: 0.9518 - val_loss: 0.0846 - val_accuracy: 0.9819
Epoch 5/10
27/27 [==============================] - 7s 263ms/step - loss: 0.1637 - accuracy: 0.9585 - val_loss: 0.0745 - val_accuracy: 0.9832
Epoch 6/10
27/27 [==============================] - 7s 254ms/step - loss: 0.1400 - accuracy: 0.9648 - val_loss: 0.0513 - val_accuracy: 0.9877
Epoch 7/10
27/27 [==============================] - 7s 255ms/step - loss: 0.1163 - accuracy: 0.9685 - val_loss: 0.0773 - val_accuracy: 0.9824
Epoc

In [37]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 9s 239ms/step - loss: 0.4646 - accuracy: 0.9159 - val_loss: 0.0664 - val_accuracy: 0.9851
Epoch 2/10
32/32 [==============================] - 7s 230ms/step - loss: 0.2364 - accuracy: 0.9443 - val_loss: 0.0766 - val_accuracy: 0.9830
Epoch 3/10
32/32 [==============================] - 7s 229ms/step - loss: 0.1691 - accuracy: 0.9557 - val_loss: 0.0522 - val_accuracy: 0.9869
Epoch 4/10
32/32 [==============================] - 7s 230ms/step - loss: 0.1255 - accuracy: 0.9719 - val_loss: 0.0490 - val_accuracy: 0.9871
Epoch 5/10
32/32 [==============================] - 7s 230ms/step - loss: 0.1250 - accuracy: 0.9687 - val_loss: 0.0587 - val_accuracy: 0.9857
Epoch 6/10
32/32 [==============================] - 7s 228ms/step - loss: 0.1152 - accuracy: 0.9739 - val_loss: 0.0741 - val_accuracy: 0.9816
Epoch 7/10
32/32 [==============================] - 7s 229ms/step - loss: 0.1381 - accuracy: 0.9698 - val_loss: 0.0518 - val_accuracy: 0.9866
Epoc

In [38]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 10s 221ms/step - loss: 0.3692 - accuracy: 0.9305 - val_loss: 0.0950 - val_accuracy: 0.9764
Epoch 2/10
38/38 [==============================] - 8s 205ms/step - loss: 0.2152 - accuracy: 0.9472 - val_loss: 0.0508 - val_accuracy: 0.9861
Epoch 3/10
38/38 [==============================] - 8s 207ms/step - loss: 0.1952 - accuracy: 0.9549 - val_loss: 0.0762 - val_accuracy: 0.9838
Epoch 4/10
38/38 [==============================] - 8s 206ms/step - loss: 0.1337 - accuracy: 0.9656 - val_loss: 0.2151 - val_accuracy: 0.9564
Epoch 5/10
38/38 [==============================] - 8s 207ms/step - loss: 0.1777 - accuracy: 0.9534 - val_loss: 0.0953 - val_accuracy: 0.9774
Epoch 6/10
38/38 [==============================] - 8s 206ms/step - loss: 0.1583 - accuracy: 0.9603 - val_loss: 0.0475 - val_accuracy: 0.9876
Epoch 7/10
38/38 [==============================] - 8s 205ms/step - loss: 0.1049 - accuracy: 0.9743 - val_loss: 0.0879 - val_accuracy: 0.9798
Epo

In [39]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 10s 201ms/step - loss: 0.3608 - accuracy: 0.9322 - val_loss: 0.0618 - val_accuracy: 0.9857
Epoch 2/10
43/43 [==============================] - 8s 193ms/step - loss: 0.1832 - accuracy: 0.9536 - val_loss: 0.0622 - val_accuracy: 0.9855
Epoch 3/10
43/43 [==============================] - 8s 194ms/step - loss: 0.1391 - accuracy: 0.9631 - val_loss: 0.0498 - val_accuracy: 0.9869
Epoch 4/10
43/43 [==============================] - 8s 193ms/step - loss: 0.1103 - accuracy: 0.9685 - val_loss: 0.0809 - val_accuracy: 0.9834
Epoch 5/10
43/43 [==============================] - 8s 193ms/step - loss: 0.1194 - accuracy: 0.9730 - val_loss: 0.0571 - val_accuracy: 0.9859
Epoch 6/10
43/43 [==============================] - 8s 194ms/step - loss: 0.1177 - accuracy: 0.9684 - val_loss: 0.0542 - val_accuracy: 0.9862
Epoch 7/10
43/43 [==============================] - 8s 194ms/step - loss: 0.1062 - accuracy: 0.9699 - val_loss: 0.0990 - val_accuracy: 0.9805
Epo

In [40]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 10s 191ms/step - loss: 0.3522 - accuracy: 0.9391 - val_loss: 0.0452 - val_accuracy: 0.9868
Epoch 2/10
48/48 [==============================] - 9s 183ms/step - loss: 0.1768 - accuracy: 0.9567 - val_loss: 0.0422 - val_accuracy: 0.9885
Epoch 3/10
48/48 [==============================] - 9s 188ms/step - loss: 0.1244 - accuracy: 0.9663 - val_loss: 0.0718 - val_accuracy: 0.9834
Epoch 4/10
48/48 [==============================] - 9s 188ms/step - loss: 0.1252 - accuracy: 0.9674 - val_loss: 0.0501 - val_accuracy: 0.9871
Epoch 5/10
48/48 [==============================] - 9s 188ms/step - loss: 0.1178 - accuracy: 0.9620 - val_loss: 0.0590 - val_accuracy: 0.9856
Epoch 6/10
48/48 [==============================] - 9s 188ms/step - loss: 0.0939 - accuracy: 0.9732 - val_loss: 0.0488 - val_accuracy: 0.9869
Epoch 7/10
48/48 [==============================] - 9s 189ms/step - loss: 0.1044 - accuracy: 0.9746 - val_loss: 0.0452 - val_accuracy: 0.9889
Epo

In [41]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 11s 182ms/step - loss: 0.2415 - accuracy: 0.9459 - val_loss: 0.0508 - val_accuracy: 0.9867
Epoch 2/10
54/54 [==============================] - 10s 180ms/step - loss: 0.1844 - accuracy: 0.9581 - val_loss: 0.0432 - val_accuracy: 0.9887
Epoch 3/10
54/54 [==============================] - 10s 179ms/step - loss: 0.1470 - accuracy: 0.9651 - val_loss: 0.0431 - val_accuracy: 0.9896
Epoch 4/10
54/54 [==============================] - 10s 182ms/step - loss: 0.1245 - accuracy: 0.9725 - val_loss: 0.1145 - val_accuracy: 0.9781
Epoch 5/10
54/54 [==============================] - 10s 181ms/step - loss: 0.1103 - accuracy: 0.9729 - val_loss: 0.0648 - val_accuracy: 0.9839
Epoch 6/10
54/54 [==============================] - 10s 182ms/step - loss: 0.1156 - accuracy: 0.9695 - val_loss: 0.0537 - val_accuracy: 0.9865
Epoch 7/10
54/54 [==============================] - 10s 183ms/step - loss: 0.0965 - accuracy: 0.9765 - val_loss: 0.0553 - val_accuracy: 0.98

In [42]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 12s 178ms/step - loss: 0.2797 - accuracy: 0.9428 - val_loss: 0.0442 - val_accuracy: 0.9879
Epoch 2/10
59/59 [==============================] - 11s 181ms/step - loss: 0.1364 - accuracy: 0.9613 - val_loss: 0.0542 - val_accuracy: 0.9862
Epoch 3/10
59/59 [==============================] - 10s 179ms/step - loss: 0.1344 - accuracy: 0.9668 - val_loss: 0.0628 - val_accuracy: 0.9831
Epoch 4/10
59/59 [==============================] - 11s 180ms/step - loss: 0.1436 - accuracy: 0.9632 - val_loss: 0.0578 - val_accuracy: 0.9870
Epoch 5/10
59/59 [==============================] - 11s 180ms/step - loss: 0.0975 - accuracy: 0.9747 - val_loss: 0.0437 - val_accuracy: 0.9878
Epoch 6/10
59/59 [==============================] - 11s 184ms/step - loss: 0.1012 - accuracy: 0.9740 - val_loss: 0.0539 - val_accuracy: 0.9870
Epoch 7/10
59/59 [==============================] - 11s 183ms/step - loss: 0.0843 - accuracy: 0.9785 - val_loss: 0.0616 - val_accuracy: 0.9

In [43]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 13s 177ms/step - loss: 0.2538 - accuracy: 0.9435 - val_loss: 0.0656 - val_accuracy: 0.9836
Epoch 2/10
64/64 [==============================] - 11s 176ms/step - loss: 0.1690 - accuracy: 0.9572 - val_loss: 0.0412 - val_accuracy: 0.9878
Epoch 3/10
64/64 [==============================] - 11s 176ms/step - loss: 0.1559 - accuracy: 0.9661 - val_loss: 0.0607 - val_accuracy: 0.9860
Epoch 4/10
64/64 [==============================] - 11s 181ms/step - loss: 0.1137 - accuracy: 0.9696 - val_loss: 0.0470 - val_accuracy: 0.9864
Epoch 5/10
64/64 [==============================] - 11s 179ms/step - loss: 0.1009 - accuracy: 0.9713 - val_loss: 0.0560 - val_accuracy: 0.9863
Epoch 6/10
64/64 [==============================] - 11s 175ms/step - loss: 0.0917 - accuracy: 0.9765 - val_loss: 0.0930 - val_accuracy: 0.9804
Epoch 7/10
64/64 [==============================] - 11s 178ms/step - loss: 0.0992 - accuracy: 0.9769 - val_loss: 0.0609 - val_accuracy: 0.9

In [44]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 13s 168ms/step - loss: 0.2112 - accuracy: 0.9540 - val_loss: 0.0684 - val_accuracy: 0.9845
Epoch 2/10
70/70 [==============================] - 12s 168ms/step - loss: 0.1423 - accuracy: 0.9611 - val_loss: 0.0644 - val_accuracy: 0.9834
Epoch 3/10
70/70 [==============================] - 12s 168ms/step - loss: 0.1509 - accuracy: 0.9593 - val_loss: 0.0824 - val_accuracy: 0.9814
Epoch 4/10
70/70 [==============================] - 12s 168ms/step - loss: 0.1343 - accuracy: 0.9681 - val_loss: 0.0553 - val_accuracy: 0.9843
Epoch 5/10
70/70 [==============================] - 12s 169ms/step - loss: 0.1032 - accuracy: 0.9719 - val_loss: 0.0463 - val_accuracy: 0.9881
Epoch 6/10
70/70 [==============================] - 12s 169ms/step - loss: 0.0978 - accuracy: 0.9729 - val_loss: 0.0600 - val_accuracy: 0.9839
Epoch 7/10
70/70 [==============================] - 12s 170ms/step - loss: 0.0788 - accuracy: 0.9798 - val_loss: 0.0426 - val_accuracy: 0.9

In [45]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 14s 165ms/step - loss: 0.2376 - accuracy: 0.9511 - val_loss: 0.1285 - val_accuracy: 0.9744
Epoch 2/10
75/75 [==============================] - 12s 165ms/step - loss: 0.1616 - accuracy: 0.9602 - val_loss: 0.0707 - val_accuracy: 0.9829
Epoch 3/10
75/75 [==============================] - 12s 165ms/step - loss: 0.1128 - accuracy: 0.9740 - val_loss: 0.0495 - val_accuracy: 0.9877
Epoch 4/10
75/75 [==============================] - 12s 164ms/step - loss: 0.1059 - accuracy: 0.9742 - val_loss: 0.0747 - val_accuracy: 0.9813
Epoch 5/10
75/75 [==============================] - 12s 165ms/step - loss: 0.1242 - accuracy: 0.9707 - val_loss: 0.0639 - val_accuracy: 0.9853
Epoch 6/10
75/75 [==============================] - 12s 165ms/step - loss: 0.1013 - accuracy: 0.9769 - val_loss: 0.0622 - val_accuracy: 0.9864
Epoch 7/10
75/75 [==============================] - 12s 167ms/step - loss: 0.0929 - accuracy: 0.9760 - val_loss: 0.0554 - val_accuracy: 0.9

In [46]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 14s 160ms/step - loss: 0.2513 - accuracy: 0.9526 - val_loss: 0.0410 - val_accuracy: 0.9888
Epoch 2/10
80/80 [==============================] - 13s 160ms/step - loss: 0.1374 - accuracy: 0.9636 - val_loss: 0.0533 - val_accuracy: 0.9858
Epoch 3/10
80/80 [==============================] - 13s 161ms/step - loss: 0.1232 - accuracy: 0.9651 - val_loss: 0.0421 - val_accuracy: 0.9876
Epoch 4/10
80/80 [==============================] - 13s 161ms/step - loss: 0.1176 - accuracy: 0.9722 - val_loss: 0.1739 - val_accuracy: 0.9834
Epoch 5/10
80/80 [==============================] - 13s 161ms/step - loss: 0.1190 - accuracy: 0.9705 - val_loss: 0.0526 - val_accuracy: 0.9857
Epoch 6/10
80/80 [==============================] - 13s 162ms/step - loss: 0.0937 - accuracy: 0.9768 - val_loss: 0.0655 - val_accuracy: 0.9854
Epoch 7/10
80/80 [==============================] - 13s 161ms/step - loss: 0.0827 - accuracy: 0.9769 - val_loss: 0.0492 - val_accuracy: 0.9

In [47]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 15s 156ms/step - loss: 0.2121 - accuracy: 0.9529 - val_loss: 0.0568 - val_accuracy: 0.9864
Epoch 2/10
85/85 [==============================] - 13s 157ms/step - loss: 0.1569 - accuracy: 0.9663 - val_loss: 0.0567 - val_accuracy: 0.9866
Epoch 3/10
85/85 [==============================] - 13s 156ms/step - loss: 0.1321 - accuracy: 0.9675 - val_loss: 0.0449 - val_accuracy: 0.9878
Epoch 4/10
85/85 [==============================] - 13s 156ms/step - loss: 0.1218 - accuracy: 0.9726 - val_loss: 0.0540 - val_accuracy: 0.9862
Epoch 5/10
85/85 [==============================] - 13s 157ms/step - loss: 0.1184 - accuracy: 0.9721 - val_loss: 0.0533 - val_accuracy: 0.9850
Epoch 6/10
85/85 [==============================] - 13s 155ms/step - loss: 0.1118 - accuracy: 0.9702 - val_loss: 0.0772 - val_accuracy: 0.9832
Epoch 7/10
85/85 [==============================] - 13s 158ms/step - loss: 0.1071 - accuracy: 0.9713 - val_loss: 0.0506 - val_accuracy: 0.9

In [48]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 15s 153ms/step - loss: 0.2482 - accuracy: 0.9465 - val_loss: 0.0521 - val_accuracy: 0.9874
Epoch 2/10
91/91 [==============================] - 15s 164ms/step - loss: 0.1577 - accuracy: 0.9634 - val_loss: 0.0515 - val_accuracy: 0.9865
Epoch 3/10
91/91 [==============================] - 15s 169ms/step - loss: 0.1192 - accuracy: 0.9718 - val_loss: 0.0435 - val_accuracy: 0.9889
Epoch 4/10
91/91 [==============================] - 15s 169ms/step - loss: 0.0971 - accuracy: 0.9734 - val_loss: 0.0523 - val_accuracy: 0.9861
Epoch 5/10
91/91 [==============================] - 15s 168ms/step - loss: 0.1239 - accuracy: 0.9735 - val_loss: 0.0486 - val_accuracy: 0.9871
Epoch 6/10
91/91 [==============================] - 15s 169ms/step - loss: 0.0954 - accuracy: 0.9738 - val_loss: 0.0575 - val_accuracy: 0.9855
Epoch 7/10
91/91 [==============================] - 15s 169ms/step - loss: 0.0722 - accuracy: 0.9782 - val_loss: 0.0734 - val_accuracy: 0.9

In [49]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 18s 167ms/step - loss: 0.2522 - accuracy: 0.9631 - val_loss: 0.0585 - val_accuracy: 0.9867
Epoch 2/10
96/96 [==============================] - 16s 167ms/step - loss: 0.1446 - accuracy: 0.9678 - val_loss: 0.0607 - val_accuracy: 0.9860
Epoch 3/10
96/96 [==============================] - 16s 167ms/step - loss: 0.1361 - accuracy: 0.9709 - val_loss: 0.0401 - val_accuracy: 0.9888
Epoch 4/10
96/96 [==============================] - 16s 165ms/step - loss: 0.1139 - accuracy: 0.9723 - val_loss: 0.0493 - val_accuracy: 0.9880
Epoch 5/10
96/96 [==============================] - 16s 165ms/step - loss: 0.0953 - accuracy: 0.9733 - val_loss: 0.0571 - val_accuracy: 0.9862
Epoch 6/10
96/96 [==============================] - 16s 166ms/step - loss: 0.0859 - accuracy: 0.9774 - val_loss: 0.0444 - val_accuracy: 0.9873
Epoch 7/10
96/96 [==============================] - 16s 166ms/step - loss: 0.0738 - accuracy: 0.9791 - val_loss: 0.0640 - val_accuracy: 0.9

In [50]:
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 18s 164ms/step - loss: 0.2161 - accuracy: 0.9563 - val_loss: 0.0422 - val_accuracy: 0.9877
Epoch 2/10
101/101 [==============================] - 17s 165ms/step - loss: 0.1322 - accuracy: 0.9654 - val_loss: 0.0446 - val_accuracy: 0.9888
Epoch 3/10
101/101 [==============================] - 16s 163ms/step - loss: 0.1248 - accuracy: 0.9743 - val_loss: 0.0571 - val_accuracy: 0.9871
Epoch 4/10
101/101 [==============================] - 16s 164ms/step - loss: 0.1178 - accuracy: 0.9732 - val_loss: 0.0496 - val_accuracy: 0.9853
Epoch 5/10
101/101 [==============================] - 16s 163ms/step - loss: 0.0981 - accuracy: 0.9759 - val_loss: 0.0425 - val_accuracy: 0.9877
Epoch 6/10
101/101 [==============================] - 17s 164ms/step - loss: 0.0946 - accuracy: 0.9762 - val_loss: 0.0548 - val_accuracy: 0.9857
Epoch 7/10
101/101 [==============================] - 16s 164ms/step - loss: 0.0773 - accuracy: 0.9802 - val_loss: 0.0584 - val

In [51]:
#
print(n)
models_lsa[n].fit_model(image_sets_lsa[n],label_sets_lsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 19s 160ms/step - loss: 0.1801 - accuracy: 0.9609 - val_loss: 0.0449 - val_accuracy: 0.9875
Epoch 2/10
110/110 [==============================] - 17s 158ms/step - loss: 0.1299 - accuracy: 0.9650 - val_loss: 0.0857 - val_accuracy: 0.9822
Epoch 3/10
110/110 [==============================] - 17s 159ms/step - loss: 0.1241 - accuracy: 0.9704 - val_loss: 0.0503 - val_accuracy: 0.9876
Epoch 4/10
110/110 [==============================] - 17s 158ms/step - loss: 0.1200 - accuracy: 0.9720 - val_loss: 0.0426 - val_accuracy: 0.9883
Epoch 5/10
110/110 [==============================] - 17s 159ms/step - loss: 0.0949 - accuracy: 0.9765 - val_loss: 0.0432 - val_accuracy: 0.9875
Epoch 6/10
110/110 [==============================] - 17s 158ms/step - loss: 0.0814 - accuracy: 0.9764 - val_loss: 0.0667 - val_accuracy: 0.9836
Epoch 7/10
110/110 [==============================] - 17s 159ms/step - loss: 0.0934 - accuracy: 0.9764 - val_loss: 0.0598 - val

In [52]:
#new_model_lsa_dir  = "D:/models/gtsrb_models/C3/gtsrb_model_c3_sep_lsa_e1"
new_model_lsa_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_lsa_e1"
#new_model_lsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_lsa_e1"

i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_lsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mni

In [53]:
import gc

In [54]:
try:
    del lsa_values
    del top_images_by_lsa
    del top_labels_by_lsa
    del image_sets_lsa
    del label_sets_lsa
    del models_lsa
except:
    print("Error")

In [55]:
gc.collect()

314922

## Training guided by DSA

In [56]:
dsa_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"

In [57]:
#dsa_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/dsa_values.npy"

In [58]:
dsa_values = np.load(dsa_direction)[len(x_train):] 
print(len(dsa_values))

6999


In [59]:
top_images_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),x_adversary_training)
top_labels_by_dsa = utils.get_x_of_indexes(list(np.flip(np.argsort(dsa_values))),y_adversary_training)

In [60]:
m = n_data_points
n = 0
image_sets_dsa = []
label_sets_dsa = []


for i in range((len(top_images_by_dsa)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dsa)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dsa)%m))
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m+(len(top_images_by_dsa)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dsa_n = np.array(top_images_by_dsa[:n+m])
        top_labels_by_dsa_n = np.array(top_labels_by_dsa[:n+m])
    image_sets_dsa.append(top_images_by_dsa_n)
    label_sets_dsa.append(top_labels_by_dsa_n)
    print(len(top_images_by_dsa_n))
    n += m



0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [61]:
print(model_dir)

models_dsa = []
for i in range(len(label_sets_dsa)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dsa.append(model)


C:/Users/fjdur/Documents/upc-july/models/models2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [62]:
n=0
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 6s 964ms/step - loss: 0.7587 - accuracy: 0.8145 - val_loss: 0.1668 - val_accuracy: 0.9779
Epoch 2/10
6/6 [==============================] - 5s 911ms/step - loss: 0.5281 - accuracy: 0.9065 - val_loss: 0.0533 - val_accuracy: 0.9856
Epoch 3/10
6/6 [==============================] - 5s 916ms/step - loss: 0.3028 - accuracy: 0.9154 - val_loss: 0.0500 - val_accuracy: 0.9883
Epoch 4/10
6/6 [==============================] - 5s 920ms/step - loss: 0.1708 - accuracy: 0.9455 - val_loss: 0.0845 - val_accuracy: 0.9849
Epoch 5/10
6/6 [==============================] - 5s 922ms/step - loss: 0.1713 - accuracy: 0.9487 - val_loss: 0.0678 - val_accuracy: 0.9849
Epoch 6/10
6/6 [==============================] - 5s 923ms/step - loss: 0.2010 - accuracy: 0.9595 - val_loss: 0.0642 - val_accuracy: 0.9861
Epoch 7/10
6/6 [==============================] - 5s 928ms/step - loss: 0.2988 - accuracy: 0.9128 - val_loss: 0.0638 - val_accuracy: 0.9856
Epoch 8/10
6/6 [==

In [63]:
n=1
print(n)

models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 7s 551ms/step - loss: 0.8084 - accuracy: 0.8371 - val_loss: 0.0757 - val_accuracy: 0.9849
Epoch 2/10
11/11 [==============================] - 5s 512ms/step - loss: 0.3320 - accuracy: 0.9298 - val_loss: 0.0652 - val_accuracy: 0.9844
Epoch 3/10
11/11 [==============================] - 5s 517ms/step - loss: 0.3124 - accuracy: 0.9210 - val_loss: 0.0742 - val_accuracy: 0.9826
Epoch 4/10
11/11 [==============================] - 5s 514ms/step - loss: 0.3463 - accuracy: 0.9353 - val_loss: 0.0808 - val_accuracy: 0.9849
Epoch 5/10
11/11 [==============================] - 5s 520ms/step - loss: 0.2537 - accuracy: 0.9421 - val_loss: 0.0474 - val_accuracy: 0.9872
Epoch 6/10
11/11 [==============================] - 5s 525ms/step - loss: 0.2294 - accuracy: 0.9293 - val_loss: 0.0524 - val_accuracy: 0.9879
Epoch 7/10
11/11 [==============================] - 5s 519ms/step - loss: 0.1804 - accuracy: 0.9590 - val_loss: 0.0401 - val_accuracy: 0.9889
Epoc

In [64]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 7s 398ms/step - loss: 0.5816 - accuracy: 0.8847 - val_loss: 0.0511 - val_accuracy: 0.9876
Epoch 2/10
16/16 [==============================] - 6s 377ms/step - loss: 0.4154 - accuracy: 0.9048 - val_loss: 0.0480 - val_accuracy: 0.9880
Epoch 3/10
16/16 [==============================] - 6s 377ms/step - loss: 0.2460 - accuracy: 0.9429 - val_loss: 0.0581 - val_accuracy: 0.9862
Epoch 4/10
16/16 [==============================] - 6s 378ms/step - loss: 0.2916 - accuracy: 0.9245 - val_loss: 0.0554 - val_accuracy: 0.9866
Epoch 5/10
16/16 [==============================] - 6s 377ms/step - loss: 0.2161 - accuracy: 0.9520 - val_loss: 0.0416 - val_accuracy: 0.9872
Epoch 6/10
16/16 [==============================] - 6s 379ms/step - loss: 0.1851 - accuracy: 0.9581 - val_loss: 0.0481 - val_accuracy: 0.9873
Epoch 7/10
16/16 [==============================] - 6s 378ms/step - loss: 0.2076 - accuracy: 0.9369 - val_loss: 0.0548 - val_accuracy: 0.9867
Epoc

In [65]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 8s 310ms/step - loss: 0.4464 - accuracy: 0.9035 - val_loss: 0.0619 - val_accuracy: 0.9854
Epoch 2/10
22/22 [==============================] - 6s 296ms/step - loss: 0.2770 - accuracy: 0.9460 - val_loss: 0.0682 - val_accuracy: 0.9844
Epoch 3/10
22/22 [==============================] - 6s 295ms/step - loss: 0.2873 - accuracy: 0.9335 - val_loss: 0.0581 - val_accuracy: 0.9864
Epoch 4/10
22/22 [==============================] - 6s 294ms/step - loss: 0.2048 - accuracy: 0.9508 - val_loss: 0.0681 - val_accuracy: 0.9853
Epoch 5/10
22/22 [==============================] - 6s 304ms/step - loss: 0.2398 - accuracy: 0.9360 - val_loss: 0.0642 - val_accuracy: 0.9856
Epoch 6/10
22/22 [==============================] - 6s 303ms/step - loss: 0.1168 - accuracy: 0.9640 - val_loss: 0.0741 - val_accuracy: 0.9841
Epoch 7/10
22/22 [==============================] - 6s 299ms/step - loss: 0.1286 - accuracy: 0.9627 - val_loss: 0.0587 - val_accuracy: 0.9867
Epoc

In [66]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 8s 272ms/step - loss: 0.4254 - accuracy: 0.9210 - val_loss: 0.0660 - val_accuracy: 0.9834
Epoch 2/10
27/27 [==============================] - 7s 259ms/step - loss: 0.2774 - accuracy: 0.9300 - val_loss: 0.0759 - val_accuracy: 0.9831
Epoch 3/10
27/27 [==============================] - 7s 259ms/step - loss: 0.2278 - accuracy: 0.9470 - val_loss: 0.0525 - val_accuracy: 0.9873
Epoch 4/10
27/27 [==============================] - 7s 259ms/step - loss: 0.2393 - accuracy: 0.9354 - val_loss: 0.0602 - val_accuracy: 0.9849
Epoch 5/10
27/27 [==============================] - 7s 260ms/step - loss: 0.1625 - accuracy: 0.9526 - val_loss: 0.0545 - val_accuracy: 0.9866
Epoch 6/10
27/27 [==============================] - 7s 260ms/step - loss: 0.1477 - accuracy: 0.9673 - val_loss: 0.0804 - val_accuracy: 0.9804
Epoch 7/10
27/27 [==============================] - 7s 258ms/step - loss: 0.1447 - accuracy: 0.9609 - val_loss: 0.0838 - val_accuracy: 0.9816
Epoc

In [67]:
n=5
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 9s 252ms/step - loss: 0.3915 - accuracy: 0.9100 - val_loss: 0.0679 - val_accuracy: 0.9840
Epoch 2/10
32/32 [==============================] - 7s 237ms/step - loss: 0.2460 - accuracy: 0.9503 - val_loss: 0.0574 - val_accuracy: 0.9852
Epoch 3/10
32/32 [==============================] - 7s 232ms/step - loss: 0.2086 - accuracy: 0.9500 - val_loss: 0.0466 - val_accuracy: 0.9877
Epoch 4/10
32/32 [==============================] - 7s 234ms/step - loss: 0.1596 - accuracy: 0.9663 - val_loss: 0.0475 - val_accuracy: 0.9878
Epoch 5/10
32/32 [==============================] - 7s 233ms/step - loss: 0.1618 - accuracy: 0.9545 - val_loss: 0.0590 - val_accuracy: 0.9871
Epoch 6/10
32/32 [==============================] - 7s 232ms/step - loss: 0.1377 - accuracy: 0.9622 - val_loss: 0.0531 - val_accuracy: 0.9876
Epoch 7/10
32/32 [==============================] - 7s 233ms/step - loss: 0.1289 - accuracy: 0.9677 - val_loss: 0.0630 - val_accuracy: 0.9863
Epoc

In [68]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 9s 217ms/step - loss: 0.3657 - accuracy: 0.9356 - val_loss: 0.0763 - val_accuracy: 0.9835
Epoch 2/10
38/38 [==============================] - 8s 209ms/step - loss: 0.2104 - accuracy: 0.9511 - val_loss: 0.0816 - val_accuracy: 0.9826
Epoch 3/10
38/38 [==============================] - 8s 213ms/step - loss: 0.1753 - accuracy: 0.9537 - val_loss: 0.0681 - val_accuracy: 0.9840
Epoch 4/10
38/38 [==============================] - 8s 210ms/step - loss: 0.1461 - accuracy: 0.9655 - val_loss: 0.0615 - val_accuracy: 0.9862
Epoch 5/10
38/38 [==============================] - 8s 213ms/step - loss: 0.1319 - accuracy: 0.9632 - val_loss: 0.0646 - val_accuracy: 0.9849
Epoch 6/10
38/38 [==============================] - 8s 211ms/step - loss: 0.1174 - accuracy: 0.9644 - val_loss: 0.0605 - val_accuracy: 0.9858
Epoch 7/10
38/38 [==============================] - 8s 211ms/step - loss: 0.1191 - accuracy: 0.9719 - val_loss: 0.0580 - val_accuracy: 0.9854
Epoc

In [69]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 10s 204ms/step - loss: 0.3514 - accuracy: 0.9411 - val_loss: 0.0473 - val_accuracy: 0.9881
Epoch 2/10
43/43 [==============================] - 9s 202ms/step - loss: 0.2031 - accuracy: 0.9506 - val_loss: 0.0657 - val_accuracy: 0.9837
Epoch 3/10
43/43 [==============================] - 9s 202ms/step - loss: 0.1595 - accuracy: 0.9597 - val_loss: 0.0557 - val_accuracy: 0.9869
Epoch 4/10
43/43 [==============================] - 9s 202ms/step - loss: 0.1375 - accuracy: 0.9660 - val_loss: 0.0419 - val_accuracy: 0.9890
Epoch 5/10
43/43 [==============================] - 9s 202ms/step - loss: 0.1058 - accuracy: 0.9715 - val_loss: 0.0496 - val_accuracy: 0.9881
Epoch 6/10
43/43 [==============================] - 9s 203ms/step - loss: 0.0897 - accuracy: 0.9745 - val_loss: 0.0414 - val_accuracy: 0.9878
Epoch 7/10
43/43 [==============================] - 9s 204ms/step - loss: 0.1628 - accuracy: 0.9679 - val_loss: 0.0489 - val_accuracy: 0.9873
Epo

In [70]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 11s 194ms/step - loss: 0.2598 - accuracy: 0.9430 - val_loss: 0.0752 - val_accuracy: 0.9829
Epoch 2/10
48/48 [==============================] - 9s 196ms/step - loss: 0.1723 - accuracy: 0.9592 - val_loss: 0.0857 - val_accuracy: 0.9804
Epoch 3/10
48/48 [==============================] - 9s 198ms/step - loss: 0.1299 - accuracy: 0.9664 - val_loss: 0.0500 - val_accuracy: 0.9876
Epoch 4/10
48/48 [==============================] - 9s 199ms/step - loss: 0.1246 - accuracy: 0.9703 - val_loss: 0.0521 - val_accuracy: 0.9877
Epoch 5/10
48/48 [==============================] - 9s 199ms/step - loss: 0.1203 - accuracy: 0.9658 - val_loss: 0.0781 - val_accuracy: 0.9844
Epoch 6/10
48/48 [==============================] - 9s 199ms/step - loss: 0.0746 - accuracy: 0.9796 - val_loss: 0.0657 - val_accuracy: 0.9863
Epoch 7/10
48/48 [==============================] - 10s 201ms/step - loss: 0.1136 - accuracy: 0.9713 - val_loss: 0.0520 - val_accuracy: 0.9878
Ep

In [71]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 12s 191ms/step - loss: 0.2784 - accuracy: 0.9436 - val_loss: 0.0466 - val_accuracy: 0.9867
Epoch 2/10
54/54 [==============================] - 10s 190ms/step - loss: 0.1613 - accuracy: 0.9616 - val_loss: 0.0742 - val_accuracy: 0.9846
Epoch 3/10
54/54 [==============================] - 10s 190ms/step - loss: 0.1566 - accuracy: 0.9583 - val_loss: 0.0996 - val_accuracy: 0.9774
Epoch 4/10
54/54 [==============================] - 10s 191ms/step - loss: 0.1303 - accuracy: 0.9705 - val_loss: 0.0627 - val_accuracy: 0.9866
Epoch 5/10
54/54 [==============================] - 10s 194ms/step - loss: 0.1207 - accuracy: 0.9652 - val_loss: 0.1300 - val_accuracy: 0.9702
Epoch 6/10
54/54 [==============================] - 10s 192ms/step - loss: 0.1229 - accuracy: 0.9679 - val_loss: 0.0527 - val_accuracy: 0.9863
Epoch 7/10
54/54 [==============================] - 10s 194ms/step - loss: 0.0735 - accuracy: 0.9811 - val_loss: 0.1262 - val_accuracy: 0.97

In [72]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 12s 189ms/step - loss: 0.2585 - accuracy: 0.9487 - val_loss: 0.0612 - val_accuracy: 0.9849
Epoch 2/10
59/59 [==============================] - 11s 187ms/step - loss: 0.1566 - accuracy: 0.9621 - val_loss: 0.0636 - val_accuracy: 0.9838
Epoch 3/10
59/59 [==============================] - 11s 187ms/step - loss: 0.1409 - accuracy: 0.9615 - val_loss: 0.0684 - val_accuracy: 0.9835
Epoch 4/10
59/59 [==============================] - 11s 186ms/step - loss: 0.1010 - accuracy: 0.9721 - val_loss: 0.0572 - val_accuracy: 0.9851
Epoch 5/10
59/59 [==============================] - 11s 186ms/step - loss: 0.1028 - accuracy: 0.9747 - val_loss: 0.0502 - val_accuracy: 0.9877
Epoch 6/10
59/59 [==============================] - 11s 188ms/step - loss: 0.0876 - accuracy: 0.9749 - val_loss: 0.0431 - val_accuracy: 0.9879
Epoch 7/10
59/59 [==============================] - 11s 187ms/step - loss: 0.0874 - accuracy: 0.9789 - val_loss: 0.0580 - val_accuracy: 0.9

In [73]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 13s 181ms/step - loss: 0.2695 - accuracy: 0.9505 - val_loss: 0.0537 - val_accuracy: 0.9856
Epoch 2/10
64/64 [==============================] - 11s 181ms/step - loss: 0.1698 - accuracy: 0.9645 - val_loss: 0.0509 - val_accuracy: 0.9866
Epoch 3/10
64/64 [==============================] - 12s 181ms/step - loss: 0.1285 - accuracy: 0.9713 - val_loss: 0.0431 - val_accuracy: 0.9877
Epoch 4/10
64/64 [==============================] - 11s 181ms/step - loss: 0.1083 - accuracy: 0.9749 - val_loss: 0.0545 - val_accuracy: 0.9871
Epoch 5/10
64/64 [==============================] - 11s 180ms/step - loss: 0.1208 - accuracy: 0.9728 - val_loss: 0.0863 - val_accuracy: 0.9817
Epoch 6/10
64/64 [==============================] - 12s 182ms/step - loss: 0.0981 - accuracy: 0.9768 - val_loss: 0.0617 - val_accuracy: 0.9864
Epoch 7/10
64/64 [==============================] - 11s 180ms/step - loss: 0.0686 - accuracy: 0.9818 - val_loss: 0.0718 - val_accuracy: 0.9

In [74]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 13s 173ms/step - loss: 0.2693 - accuracy: 0.9494 - val_loss: 0.0412 - val_accuracy: 0.9879
Epoch 2/10
70/70 [==============================] - 12s 172ms/step - loss: 0.1269 - accuracy: 0.9699 - val_loss: 0.0528 - val_accuracy: 0.9884
Epoch 3/10
70/70 [==============================] - 12s 172ms/step - loss: 0.1137 - accuracy: 0.9722 - val_loss: 0.0785 - val_accuracy: 0.9842
Epoch 4/10
70/70 [==============================] - 12s 172ms/step - loss: 0.1143 - accuracy: 0.9730 - val_loss: 0.0962 - val_accuracy: 0.9758
Epoch 5/10
70/70 [==============================] - 12s 172ms/step - loss: 0.1199 - accuracy: 0.9693 - val_loss: 0.0503 - val_accuracy: 0.9873
Epoch 6/10
70/70 [==============================] - 12s 174ms/step - loss: 0.0855 - accuracy: 0.9733 - val_loss: 0.0990 - val_accuracy: 0.9789
Epoch 7/10
70/70 [==============================] - 12s 173ms/step - loss: 0.0843 - accuracy: 0.9798 - val_loss: 0.0808 - val_accuracy: 0.9

In [75]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 14s 170ms/step - loss: 0.2616 - accuracy: 0.9434 - val_loss: 0.0494 - val_accuracy: 0.9881
Epoch 2/10
75/75 [==============================] - 13s 168ms/step - loss: 0.1583 - accuracy: 0.9663 - val_loss: 0.0557 - val_accuracy: 0.9857
Epoch 3/10
75/75 [==============================] - 13s 169ms/step - loss: 0.1412 - accuracy: 0.9684 - val_loss: 0.0566 - val_accuracy: 0.9841
Epoch 4/10
75/75 [==============================] - 13s 169ms/step - loss: 0.1081 - accuracy: 0.9731 - val_loss: 0.0569 - val_accuracy: 0.9858
Epoch 5/10
75/75 [==============================] - 13s 169ms/step - loss: 0.0953 - accuracy: 0.9758 - val_loss: 0.0858 - val_accuracy: 0.9828
Epoch 6/10
75/75 [==============================] - 13s 170ms/step - loss: 0.1188 - accuracy: 0.9722 - val_loss: 0.0504 - val_accuracy: 0.9869
Epoch 7/10
75/75 [==============================] - 13s 170ms/step - loss: 0.0818 - accuracy: 0.9772 - val_loss: 0.0521 - val_accuracy: 0.9

In [76]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 14s 166ms/step - loss: 0.2413 - accuracy: 0.9576 - val_loss: 0.0799 - val_accuracy: 0.9846
Epoch 2/10
80/80 [==============================] - 13s 163ms/step - loss: 0.1374 - accuracy: 0.9680 - val_loss: 0.0617 - val_accuracy: 0.9841
Epoch 3/10
80/80 [==============================] - 13s 164ms/step - loss: 0.1312 - accuracy: 0.9701 - val_loss: 0.0551 - val_accuracy: 0.9878
Epoch 4/10
80/80 [==============================] - 13s 164ms/step - loss: 0.1206 - accuracy: 0.9684 - val_loss: 0.0425 - val_accuracy: 0.9881
Epoch 5/10
80/80 [==============================] - 13s 166ms/step - loss: 0.0967 - accuracy: 0.9751 - val_loss: 0.0703 - val_accuracy: 0.9853
Epoch 6/10
80/80 [==============================] - 13s 166ms/step - loss: 0.1166 - accuracy: 0.9758 - val_loss: 0.0658 - val_accuracy: 0.9844
Epoch 7/10
80/80 [==============================] - 13s 165ms/step - loss: 0.0810 - accuracy: 0.9783 - val_loss: 0.0686 - val_accuracy: 0.9

In [77]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 15s 165ms/step - loss: 0.2400 - accuracy: 0.9560 - val_loss: 0.0449 - val_accuracy: 0.9885
Epoch 2/10
85/85 [==============================] - 14s 162ms/step - loss: 0.1320 - accuracy: 0.9658 - val_loss: 0.0425 - val_accuracy: 0.9879
Epoch 3/10
85/85 [==============================] - 14s 160ms/step - loss: 0.1283 - accuracy: 0.9697 - val_loss: 0.0540 - val_accuracy: 0.9882
Epoch 4/10
85/85 [==============================] - 14s 163ms/step - loss: 0.0967 - accuracy: 0.9734 - val_loss: 0.0710 - val_accuracy: 0.9848
Epoch 5/10
85/85 [==============================] - 14s 161ms/step - loss: 0.1071 - accuracy: 0.9757 - val_loss: 0.0559 - val_accuracy: 0.9876
Epoch 6/10
85/85 [==============================] - 14s 162ms/step - loss: 0.1060 - accuracy: 0.9706 - val_loss: 0.0566 - val_accuracy: 0.9875
Epoch 7/10
85/85 [==============================] - 14s 162ms/step - loss: 0.0898 - accuracy: 0.9787 - val_loss: 0.0578 - val_accuracy: 0.9

In [78]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 16s 161ms/step - loss: 0.1972 - accuracy: 0.9556 - val_loss: 0.0907 - val_accuracy: 0.9834
Epoch 2/10
91/91 [==============================] - 16s 176ms/step - loss: 0.1475 - accuracy: 0.9691 - val_loss: 0.1113 - val_accuracy: 0.9798
Epoch 3/10
91/91 [==============================] - 16s 174ms/step - loss: 0.1119 - accuracy: 0.9680 - val_loss: 0.0520 - val_accuracy: 0.9876
Epoch 4/10
91/91 [==============================] - 16s 175ms/step - loss: 0.1081 - accuracy: 0.9720 - val_loss: 0.0582 - val_accuracy: 0.9872
Epoch 5/10
91/91 [==============================] - 16s 177ms/step - loss: 0.1130 - accuracy: 0.9716 - val_loss: 0.0478 - val_accuracy: 0.9886
Epoch 6/10
91/91 [==============================] - 17s 185ms/step - loss: 0.0859 - accuracy: 0.9773 - val_loss: 0.0797 - val_accuracy: 0.9796
Epoch 7/10
91/91 [==============================] - 17s 187ms/step - loss: 0.0803 - accuracy: 0.9770 - val_loss: 0.0446 - val_accuracy: 0.9

In [79]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 18s 175ms/step - loss: 0.1929 - accuracy: 0.9607 - val_loss: 0.0597 - val_accuracy: 0.9859
Epoch 2/10
96/96 [==============================] - 17s 173ms/step - loss: 0.1380 - accuracy: 0.9689 - val_loss: 0.0748 - val_accuracy: 0.9849
Epoch 3/10
96/96 [==============================] - 17s 173ms/step - loss: 0.1248 - accuracy: 0.9689 - val_loss: 0.0557 - val_accuracy: 0.9878
Epoch 4/10
96/96 [==============================] - 17s 174ms/step - loss: 0.0976 - accuracy: 0.9745 - val_loss: 0.0526 - val_accuracy: 0.9876
Epoch 5/10
96/96 [==============================] - 17s 174ms/step - loss: 0.1113 - accuracy: 0.9740 - val_loss: 0.0504 - val_accuracy: 0.9876
Epoch 6/10
96/96 [==============================] - 17s 176ms/step - loss: 0.0876 - accuracy: 0.9781 - val_loss: 0.0555 - val_accuracy: 0.9867
Epoch 7/10
96/96 [==============================] - 17s 175ms/step - loss: 0.0820 - accuracy: 0.9762 - val_loss: 0.1642 - val_accuracy: 0.9

In [80]:
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 19s 171ms/step - loss: 0.1876 - accuracy: 0.9557 - val_loss: 0.0553 - val_accuracy: 0.9872
Epoch 2/10
101/101 [==============================] - 17s 172ms/step - loss: 0.1337 - accuracy: 0.9690 - val_loss: 0.0603 - val_accuracy: 0.9856
Epoch 3/10
101/101 [==============================] - 17s 171ms/step - loss: 0.1277 - accuracy: 0.9716 - val_loss: 0.0524 - val_accuracy: 0.9877
Epoch 4/10
101/101 [==============================] - 17s 172ms/step - loss: 0.0867 - accuracy: 0.9773 - val_loss: 0.0535 - val_accuracy: 0.9869
Epoch 5/10
101/101 [==============================] - 17s 170ms/step - loss: 0.0998 - accuracy: 0.9764 - val_loss: 0.0989 - val_accuracy: 0.9815
Epoch 6/10
101/101 [==============================] - 17s 172ms/step - loss: 0.0854 - accuracy: 0.9781 - val_loss: 0.0577 - val_accuracy: 0.9841
Epoch 7/10
101/101 [==============================] - 17s 171ms/step - loss: 0.0805 - accuracy: 0.9796 - val_loss: 0.0604 - val

In [81]:
#
print(n)
models_dsa[n].fit_model(image_sets_dsa[n],label_sets_dsa[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 21s 168ms/step - loss: 0.2089 - accuracy: 0.9620 - val_loss: 0.0589 - val_accuracy: 0.9843
Epoch 2/10
110/110 [==============================] - 18s 168ms/step - loss: 0.1748 - accuracy: 0.9621 - val_loss: 0.0582 - val_accuracy: 0.9842
Epoch 3/10
110/110 [==============================] - 18s 168ms/step - loss: 0.1124 - accuracy: 0.9746 - val_loss: 0.0580 - val_accuracy: 0.9876
Epoch 4/10
110/110 [==============================] - 18s 169ms/step - loss: 0.1287 - accuracy: 0.9741 - val_loss: 0.0577 - val_accuracy: 0.9854
Epoch 5/10
110/110 [==============================] - 18s 169ms/step - loss: 0.0780 - accuracy: 0.9787 - val_loss: 0.0549 - val_accuracy: 0.9866
Epoch 6/10
110/110 [==============================] - 18s 168ms/step - loss: 0.0885 - accuracy: 0.9739 - val_loss: 0.0616 - val_accuracy: 0.9848
Epoch 7/10
110/110 [==============================] - 19s 173ms/step - loss: 0.0873 - accuracy: 0.9777 - val_loss: 0.0732 - val

In [82]:
#new_model_dsa_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_dsa_e1"
new_model_dsa_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_dsa_e1"


i=0

for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dsa_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mni

In [83]:
try:
    del dsa_values
    del top_images_by_dsa
    del top_labels_by_dsa
    del image_sets_dsa
    del label_sets_dsa
    del models_dsa
except:
    print("Error")

In [84]:
import gc
gc.collect()

248212

## Uncertainty

In [85]:
#dg_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/deep_gini_values.npy"

#deep_gini_values = np.load(dg_direction)
dg_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"

deep_gini_values = np.load(dg_direction)[len(x_train):] 
print(len(deep_gini_values))

6999


In [86]:
# Obtaining top n images by dg values
top_images_by_dg  = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),x_adversary_training)
top_labels_by_dg = utils.get_x_of_indexes(list(np.flip(np.argsort(deep_gini_values))),y_adversary_training)

In [87]:
m = n_data_points
n = 0
image_sets_dg = []
label_sets_dg = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_dg)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_dg)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_dg)%m))
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m+(len(top_images_by_dg)%m)])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m+(len(top_images_by_dg)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_dg_n = np.array(top_images_by_dg[:n+m])
        top_labels_by_dg_n = np.array(top_labels_by_dg[:n+m])
    image_sets_dg.append(top_images_by_dg_n)
    label_sets_dg.append(top_labels_by_dg_n)
    print(len(top_images_by_dg_n))
    n += m



0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [88]:
print(model_dir)
models_dg = []
#for i in range(len(label_sets_lsa)):
for i in range(len(label_sets_dg)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_dg.append(model)

C:/Users/fjdur/Documents/upc-july/models/models2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [89]:
n=0
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 6s 1s/step - loss: 0.7633 - accuracy: 0.8406 - val_loss: 0.0711 - val_accuracy: 0.9865
Epoch 2/10
6/6 [==============================] - 5s 977ms/step - loss: 0.3051 - accuracy: 0.9462 - val_loss: 0.0682 - val_accuracy: 0.9869
Epoch 3/10
6/6 [==============================] - 5s 978ms/step - loss: 0.1745 - accuracy: 0.9546 - val_loss: 0.0853 - val_accuracy: 0.9844
Epoch 4/10
6/6 [==============================] - 5s 995ms/step - loss: 0.1246 - accuracy: 0.9604 - val_loss: 0.0806 - val_accuracy: 0.9846
Epoch 5/10
6/6 [==============================] - 6s 1s/step - loss: 0.1679 - accuracy: 0.9493 - val_loss: 0.1045 - val_accuracy: 0.9836
Epoch 6/10
6/6 [==============================] - 6s 1s/step - loss: 0.1302 - accuracy: 0.9480 - val_loss: 0.1026 - val_accuracy: 0.9834
Epoch 7/10
6/6 [==============================] - 6s 1s/step - loss: 0.0793 - accuracy: 0.9754 - val_loss: 0.1001 - val_accuracy: 0.9836
Epoch 8/10
6/6 [==============

In [90]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 7s 620ms/step - loss: 0.7017 - accuracy: 0.8863 - val_loss: 0.0552 - val_accuracy: 0.9866
Epoch 2/10
11/11 [==============================] - 6s 616ms/step - loss: 0.3573 - accuracy: 0.9195 - val_loss: 0.0561 - val_accuracy: 0.9871
Epoch 3/10
11/11 [==============================] - 6s 619ms/step - loss: 0.2810 - accuracy: 0.9243 - val_loss: 0.0570 - val_accuracy: 0.9862
Epoch 4/10
11/11 [==============================] - 6s 615ms/step - loss: 0.2059 - accuracy: 0.9573 - val_loss: 0.0741 - val_accuracy: 0.9861
Epoch 5/10
11/11 [==============================] - 6s 617ms/step - loss: 0.2560 - accuracy: 0.9416 - val_loss: 0.0555 - val_accuracy: 0.9874
Epoch 6/10
11/11 [==============================] - 6s 611ms/step - loss: 0.1637 - accuracy: 0.9512 - val_loss: 0.0679 - val_accuracy: 0.9864
Epoch 7/10
11/11 [==============================] - 6s 614ms/step - loss: 0.1379 - accuracy: 0.9570 - val_loss: 0.0927 - val_accuracy: 0.9820
Epoc

In [91]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 8s 439ms/step - loss: 0.6392 - accuracy: 0.8630 - val_loss: 0.0472 - val_accuracy: 0.9880
Epoch 2/10
16/16 [==============================] - 7s 443ms/step - loss: 0.3995 - accuracy: 0.8976 - val_loss: 0.0572 - val_accuracy: 0.9880
Epoch 3/10
16/16 [==============================] - 7s 445ms/step - loss: 0.2161 - accuracy: 0.9399 - val_loss: 0.0657 - val_accuracy: 0.9864
Epoch 4/10
16/16 [==============================] - 7s 449ms/step - loss: 0.3226 - accuracy: 0.9257 - val_loss: 0.0619 - val_accuracy: 0.9857
Epoch 5/10
16/16 [==============================] - 7s 441ms/step - loss: 0.2011 - accuracy: 0.9403 - val_loss: 0.0518 - val_accuracy: 0.9886
Epoch 6/10
16/16 [==============================] - 7s 446ms/step - loss: 0.2479 - accuracy: 0.9354 - val_loss: 0.0523 - val_accuracy: 0.9881
Epoch 7/10
16/16 [==============================] - 7s 446ms/step - loss: 0.1788 - accuracy: 0.9527 - val_loss: 0.0558 - val_accuracy: 0.9868
Epoc

In [92]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 8s 341ms/step - loss: 0.6211 - accuracy: 0.8952 - val_loss: 0.0450 - val_accuracy: 0.9886
Epoch 2/10
22/22 [==============================] - 7s 344ms/step - loss: 0.3020 - accuracy: 0.9116 - val_loss: 0.0635 - val_accuracy: 0.9868
Epoch 3/10
22/22 [==============================] - 7s 343ms/step - loss: 0.2868 - accuracy: 0.9279 - val_loss: 0.0460 - val_accuracy: 0.9881
Epoch 4/10
22/22 [==============================] - 7s 341ms/step - loss: 0.2509 - accuracy: 0.9372 - val_loss: 0.0569 - val_accuracy: 0.9873
Epoch 5/10
22/22 [==============================] - 8s 358ms/step - loss: 0.2429 - accuracy: 0.9364 - val_loss: 0.0540 - val_accuracy: 0.9879
Epoch 6/10
22/22 [==============================] - 7s 342ms/step - loss: 0.1927 - accuracy: 0.9517 - val_loss: 0.0672 - val_accuracy: 0.9852
Epoch 7/10
22/22 [==============================] - 7s 340ms/step - loss: 0.1699 - accuracy: 0.9474 - val_loss: 0.0663 - val_accuracy: 0.9865
Epoc

In [93]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 9s 295ms/step - loss: 0.5482 - accuracy: 0.8852 - val_loss: 0.0369 - val_accuracy: 0.9894
Epoch 2/10
27/27 [==============================] - 8s 300ms/step - loss: 0.3177 - accuracy: 0.9259 - val_loss: 0.0586 - val_accuracy: 0.9872
Epoch 3/10
27/27 [==============================] - 8s 308ms/step - loss: 0.2730 - accuracy: 0.9239 - val_loss: 0.0401 - val_accuracy: 0.9885
Epoch 4/10
27/27 [==============================] - 8s 299ms/step - loss: 0.2282 - accuracy: 0.9426 - val_loss: 0.0667 - val_accuracy: 0.9849
Epoch 5/10
27/27 [==============================] - 8s 298ms/step - loss: 0.1864 - accuracy: 0.9489 - val_loss: 0.0530 - val_accuracy: 0.9860
Epoch 6/10
27/27 [==============================] - 8s 295ms/step - loss: 0.1554 - accuracy: 0.9546 - val_loss: 0.0498 - val_accuracy: 0.9861
Epoch 7/10
27/27 [==============================] - 8s 297ms/step - loss: 0.1604 - accuracy: 0.9521 - val_loss: 0.0533 - val_accuracy: 0.9872
Epoc

In [94]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 10s 267ms/step - loss: 0.4959 - accuracy: 0.8953 - val_loss: 0.0513 - val_accuracy: 0.9880
Epoch 2/10
32/32 [==============================] - 8s 267ms/step - loss: 0.2690 - accuracy: 0.9247 - val_loss: 0.0595 - val_accuracy: 0.9868
Epoch 3/10
32/32 [==============================] - 8s 266ms/step - loss: 0.2416 - accuracy: 0.9480 - val_loss: 0.0529 - val_accuracy: 0.9874
Epoch 4/10
32/32 [==============================] - 8s 267ms/step - loss: 0.1917 - accuracy: 0.9457 - val_loss: 0.0447 - val_accuracy: 0.9878
Epoch 5/10
32/32 [==============================] - 8s 270ms/step - loss: 0.1656 - accuracy: 0.9542 - val_loss: 0.0438 - val_accuracy: 0.9888
Epoch 6/10
32/32 [==============================] - 8s 265ms/step - loss: 0.1918 - accuracy: 0.9454 - val_loss: 0.0451 - val_accuracy: 0.9879
Epoch 7/10
32/32 [==============================] - 8s 266ms/step - loss: 0.1499 - accuracy: 0.9563 - val_loss: 0.0473 - val_accuracy: 0.9889
Epo

In [95]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 10s 239ms/step - loss: 0.4581 - accuracy: 0.9213 - val_loss: 0.0688 - val_accuracy: 0.9853
Epoch 2/10
38/38 [==============================] - 9s 239ms/step - loss: 0.2942 - accuracy: 0.9253 - val_loss: 0.0467 - val_accuracy: 0.9876
Epoch 3/10
38/38 [==============================] - 9s 240ms/step - loss: 0.2711 - accuracy: 0.9358 - val_loss: 0.0741 - val_accuracy: 0.9845
Epoch 4/10
38/38 [==============================] - 9s 241ms/step - loss: 0.2299 - accuracy: 0.9405 - val_loss: 0.0612 - val_accuracy: 0.9854
Epoch 5/10
38/38 [==============================] - 9s 241ms/step - loss: 0.2160 - accuracy: 0.9454 - val_loss: 0.0635 - val_accuracy: 0.9819
Epoch 6/10
38/38 [==============================] - 9s 240ms/step - loss: 0.2385 - accuracy: 0.9330 - val_loss: 0.0761 - val_accuracy: 0.9841
Epoch 7/10
38/38 [==============================] - 9s 240ms/step - loss: 0.2341 - accuracy: 0.9472 - val_loss: 0.0499 - val_accuracy: 0.9875
Epo

In [96]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 11s 226ms/step - loss: 0.3997 - accuracy: 0.9209 - val_loss: 0.0417 - val_accuracy: 0.9879
Epoch 2/10
43/43 [==============================] - 10s 227ms/step - loss: 0.2797 - accuracy: 0.9285 - val_loss: 0.0540 - val_accuracy: 0.9877
Epoch 3/10
43/43 [==============================] - 10s 226ms/step - loss: 0.2369 - accuracy: 0.9347 - val_loss: 0.0480 - val_accuracy: 0.9868
Epoch 4/10
43/43 [==============================] - 10s 226ms/step - loss: 0.2400 - accuracy: 0.9331 - val_loss: 0.0575 - val_accuracy: 0.9861
Epoch 5/10
43/43 [==============================] - 10s 225ms/step - loss: 0.2483 - accuracy: 0.9352 - val_loss: 0.0421 - val_accuracy: 0.9889
Epoch 6/10
43/43 [==============================] - 10s 225ms/step - loss: 0.1529 - accuracy: 0.9507 - val_loss: 0.0442 - val_accuracy: 0.9884
Epoch 7/10
43/43 [==============================] - 10s 238ms/step - loss: 0.1689 - accuracy: 0.9553 - val_loss: 0.0644 - val_accuracy: 0.98

In [97]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 12s 218ms/step - loss: 0.4571 - accuracy: 0.9231 - val_loss: 0.0489 - val_accuracy: 0.9869
Epoch 2/10
48/48 [==============================] - 10s 215ms/step - loss: 0.2925 - accuracy: 0.9323 - val_loss: 0.0511 - val_accuracy: 0.9856
Epoch 3/10
48/48 [==============================] - 10s 215ms/step - loss: 0.1936 - accuracy: 0.9537 - val_loss: 0.0465 - val_accuracy: 0.9876
Epoch 4/10
48/48 [==============================] - 10s 215ms/step - loss: 0.2031 - accuracy: 0.9533 - val_loss: 0.0519 - val_accuracy: 0.9873
Epoch 5/10
48/48 [==============================] - 10s 215ms/step - loss: 0.1819 - accuracy: 0.9563 - val_loss: 0.0427 - val_accuracy: 0.9882
Epoch 6/10
48/48 [==============================] - 11s 236ms/step - loss: 0.1663 - accuracy: 0.9621 - val_loss: 0.0596 - val_accuracy: 0.9852
Epoch 7/10
48/48 [==============================] - 11s 230ms/step - loss: 0.1485 - accuracy: 0.9613 - val_loss: 0.0449 - val_accuracy: 0.98

In [98]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 12s 200ms/step - loss: 0.4069 - accuracy: 0.9168 - val_loss: 0.0731 - val_accuracy: 0.9870
Epoch 2/10
54/54 [==============================] - 11s 202ms/step - loss: 0.2755 - accuracy: 0.9389 - val_loss: 0.0540 - val_accuracy: 0.9868
Epoch 3/10
54/54 [==============================] - 11s 199ms/step - loss: 0.1998 - accuracy: 0.9494 - val_loss: 0.0404 - val_accuracy: 0.9889
Epoch 4/10
54/54 [==============================] - 11s 200ms/step - loss: 0.1525 - accuracy: 0.9567 - val_loss: 0.0456 - val_accuracy: 0.9876
Epoch 5/10
54/54 [==============================] - 11s 200ms/step - loss: 0.1710 - accuracy: 0.9555 - val_loss: 0.0487 - val_accuracy: 0.9878
Epoch 6/10
54/54 [==============================] - 11s 199ms/step - loss: 0.1554 - accuracy: 0.9588 - val_loss: 0.0716 - val_accuracy: 0.9805
Epoch 7/10
54/54 [==============================] - 11s 199ms/step - loss: 0.1536 - accuracy: 0.9585 - val_loss: 0.0516 - val_accuracy: 0.98

In [99]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 13s 193ms/step - loss: 0.3488 - accuracy: 0.9263 - val_loss: 0.0546 - val_accuracy: 0.9864
Epoch 2/10
59/59 [==============================] - 11s 194ms/step - loss: 0.2409 - accuracy: 0.9507 - val_loss: 0.0414 - val_accuracy: 0.9886
Epoch 3/10
59/59 [==============================] - 11s 195ms/step - loss: 0.2096 - accuracy: 0.9460 - val_loss: 0.0428 - val_accuracy: 0.9888
Epoch 4/10
59/59 [==============================] - 11s 193ms/step - loss: 0.2039 - accuracy: 0.9512 - val_loss: 0.0459 - val_accuracy: 0.9877
Epoch 5/10
59/59 [==============================] - 11s 194ms/step - loss: 0.1565 - accuracy: 0.9629 - val_loss: 0.0497 - val_accuracy: 0.9884
Epoch 6/10
59/59 [==============================] - 11s 193ms/step - loss: 0.1289 - accuracy: 0.9661 - val_loss: 0.0478 - val_accuracy: 0.9881
Epoch 7/10
59/59 [==============================] - 11s 193ms/step - loss: 0.1228 - accuracy: 0.9637 - val_loss: 0.0471 - val_accuracy: 0.9

In [100]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 13s 188ms/step - loss: 0.3628 - accuracy: 0.9348 - val_loss: 0.0731 - val_accuracy: 0.9849
Epoch 2/10
64/64 [==============================] - 12s 186ms/step - loss: 0.2413 - accuracy: 0.9383 - val_loss: 0.0576 - val_accuracy: 0.9838
Epoch 3/10
64/64 [==============================] - 12s 187ms/step - loss: 0.1762 - accuracy: 0.9544 - val_loss: 0.0385 - val_accuracy: 0.9890
Epoch 4/10
64/64 [==============================] - 12s 185ms/step - loss: 0.1716 - accuracy: 0.9602 - val_loss: 0.0370 - val_accuracy: 0.9902
Epoch 5/10
64/64 [==============================] - 12s 186ms/step - loss: 0.1399 - accuracy: 0.9633 - val_loss: 0.0416 - val_accuracy: 0.9881
Epoch 6/10
64/64 [==============================] - 12s 186ms/step - loss: 0.1159 - accuracy: 0.9703 - val_loss: 0.0417 - val_accuracy: 0.9876
Epoch 7/10
64/64 [==============================] - 12s 186ms/step - loss: 0.1159 - accuracy: 0.9689 - val_loss: 0.0489 - val_accuracy: 0.9

In [101]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 14s 180ms/step - loss: 0.2717 - accuracy: 0.9433 - val_loss: 0.0393 - val_accuracy: 0.9881
Epoch 2/10
70/70 [==============================] - 12s 178ms/step - loss: 0.2185 - accuracy: 0.9493 - val_loss: 0.0453 - val_accuracy: 0.9878
Epoch 3/10
70/70 [==============================] - 13s 180ms/step - loss: 0.1611 - accuracy: 0.9608 - val_loss: 0.0785 - val_accuracy: 0.9859
Epoch 4/10
70/70 [==============================] - 12s 179ms/step - loss: 0.1746 - accuracy: 0.9557 - val_loss: 0.0539 - val_accuracy: 0.9871
Epoch 5/10
70/70 [==============================] - 13s 180ms/step - loss: 0.1269 - accuracy: 0.9681 - val_loss: 0.0522 - val_accuracy: 0.9866
Epoch 6/10
70/70 [==============================] - 13s 180ms/step - loss: 0.1179 - accuracy: 0.9661 - val_loss: 0.0500 - val_accuracy: 0.9882
Epoch 7/10
70/70 [==============================] - 12s 179ms/step - loss: 0.1086 - accuracy: 0.9696 - val_loss: 0.0438 - val_accuracy: 0.9

In [102]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 15s 178ms/step - loss: 0.3054 - accuracy: 0.9400 - val_loss: 0.0518 - val_accuracy: 0.9868
Epoch 2/10
75/75 [==============================] - 13s 176ms/step - loss: 0.1845 - accuracy: 0.9520 - val_loss: 0.0528 - val_accuracy: 0.9856
Epoch 3/10
75/75 [==============================] - 13s 175ms/step - loss: 0.1617 - accuracy: 0.9553 - val_loss: 0.0679 - val_accuracy: 0.9859
Epoch 4/10
75/75 [==============================] - 13s 175ms/step - loss: 0.1406 - accuracy: 0.9649 - val_loss: 0.0700 - val_accuracy: 0.9844
Epoch 5/10
75/75 [==============================] - 13s 175ms/step - loss: 0.1568 - accuracy: 0.9654 - val_loss: 0.0527 - val_accuracy: 0.9879
Epoch 6/10
75/75 [==============================] - 13s 175ms/step - loss: 0.1399 - accuracy: 0.9640 - val_loss: 0.0584 - val_accuracy: 0.9856
Epoch 7/10
75/75 [==============================] - 13s 176ms/step - loss: 0.1061 - accuracy: 0.9694 - val_loss: 0.0514 - val_accuracy: 0.9

In [103]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 15s 176ms/step - loss: 0.2967 - accuracy: 0.9352 - val_loss: 0.0414 - val_accuracy: 0.9882
Epoch 2/10
80/80 [==============================] - 14s 173ms/step - loss: 0.1553 - accuracy: 0.9558 - val_loss: 0.0500 - val_accuracy: 0.9858
Epoch 3/10
80/80 [==============================] - 14s 174ms/step - loss: 0.1469 - accuracy: 0.9608 - val_loss: 0.0539 - val_accuracy: 0.9877
Epoch 4/10
80/80 [==============================] - 14s 173ms/step - loss: 0.1171 - accuracy: 0.9698 - val_loss: 0.0453 - val_accuracy: 0.9862
Epoch 5/10
80/80 [==============================] - 14s 172ms/step - loss: 0.1238 - accuracy: 0.9693 - val_loss: 0.0477 - val_accuracy: 0.9876
Epoch 6/10
80/80 [==============================] - 14s 173ms/step - loss: 0.1039 - accuracy: 0.9726 - val_loss: 0.0548 - val_accuracy: 0.9874
Epoch 7/10
80/80 [==============================] - 14s 172ms/step - loss: 0.1082 - accuracy: 0.9710 - val_loss: 0.0460 - val_accuracy: 0.9

In [104]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 16s 170ms/step - loss: 0.3288 - accuracy: 0.9429 - val_loss: 0.0425 - val_accuracy: 0.9879
Epoch 2/10
85/85 [==============================] - 16s 187ms/step - loss: 0.1732 - accuracy: 0.9582 - val_loss: 0.0525 - val_accuracy: 0.9859
Epoch 3/10
85/85 [==============================] - 16s 186ms/step - loss: 0.1468 - accuracy: 0.9606 - val_loss: 0.0587 - val_accuracy: 0.9849
Epoch 4/10
85/85 [==============================] - 16s 186ms/step - loss: 0.1573 - accuracy: 0.9645 - val_loss: 0.0535 - val_accuracy: 0.9868
Epoch 5/10
85/85 [==============================] - 16s 187ms/step - loss: 0.1132 - accuracy: 0.9689 - val_loss: 0.0473 - val_accuracy: 0.9861
Epoch 6/10
85/85 [==============================] - 16s 187ms/step - loss: 0.0848 - accuracy: 0.9788 - val_loss: 0.0527 - val_accuracy: 0.9873
Epoch 7/10
85/85 [==============================] - 16s 186ms/step - loss: 0.1059 - accuracy: 0.9723 - val_loss: 0.0571 - val_accuracy: 0.9

In [105]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 18s 183ms/step - loss: 0.2593 - accuracy: 0.9518 - val_loss: 0.0696 - val_accuracy: 0.9814
Epoch 2/10
91/91 [==============================] - 17s 189ms/step - loss: 0.1485 - accuracy: 0.9643 - val_loss: 0.0438 - val_accuracy: 0.9868
Epoch 3/10
91/91 [==============================] - 17s 185ms/step - loss: 0.1287 - accuracy: 0.9676 - val_loss: 0.0523 - val_accuracy: 0.9874
Epoch 4/10
91/91 [==============================] - 17s 184ms/step - loss: 0.1187 - accuracy: 0.9714 - val_loss: 0.0568 - val_accuracy: 0.9874
Epoch 5/10
91/91 [==============================] - 17s 184ms/step - loss: 0.1103 - accuracy: 0.9715 - val_loss: 0.0709 - val_accuracy: 0.9827
Epoch 6/10
91/91 [==============================] - 17s 186ms/step - loss: 0.0863 - accuracy: 0.9775 - val_loss: 0.0438 - val_accuracy: 0.9876
Epoch 7/10
91/91 [==============================] - 17s 183ms/step - loss: 0.0830 - accuracy: 0.9779 - val_loss: 0.0567 - val_accuracy: 0.9

In [106]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 19s 184ms/step - loss: 0.2504 - accuracy: 0.9525 - val_loss: 0.0509 - val_accuracy: 0.9863
Epoch 2/10
96/96 [==============================] - 17s 182ms/step - loss: 0.1647 - accuracy: 0.9632 - val_loss: 0.0530 - val_accuracy: 0.9850
Epoch 3/10
96/96 [==============================] - 17s 182ms/step - loss: 0.1342 - accuracy: 0.9666 - val_loss: 0.0783 - val_accuracy: 0.9835
Epoch 4/10
96/96 [==============================] - 17s 183ms/step - loss: 0.1154 - accuracy: 0.9700 - val_loss: 0.0541 - val_accuracy: 0.9856
Epoch 5/10
96/96 [==============================] - 17s 182ms/step - loss: 0.1083 - accuracy: 0.9722 - val_loss: 0.0526 - val_accuracy: 0.9861
Epoch 6/10
96/96 [==============================] - 17s 182ms/step - loss: 0.1105 - accuracy: 0.9722 - val_loss: 0.0511 - val_accuracy: 0.9873
Epoch 7/10
96/96 [==============================] - 17s 181ms/step - loss: 0.0972 - accuracy: 0.9760 - val_loss: 0.0432 - val_accuracy: 0.9

In [107]:
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 20s 181ms/step - loss: 0.2254 - accuracy: 0.9532 - val_loss: 0.0470 - val_accuracy: 0.9867
Epoch 2/10
101/101 [==============================] - 18s 181ms/step - loss: 0.1299 - accuracy: 0.9669 - val_loss: 0.0433 - val_accuracy: 0.9884
Epoch 3/10
101/101 [==============================] - 22s 224ms/step - loss: 0.1395 - accuracy: 0.9619 - val_loss: 0.0476 - val_accuracy: 0.9874
Epoch 4/10
101/101 [==============================] - 17s 170ms/step - loss: 0.1064 - accuracy: 0.9680 - val_loss: 0.0420 - val_accuracy: 0.9890
Epoch 5/10
101/101 [==============================] - 18s 178ms/step - loss: 0.1082 - accuracy: 0.9709 - val_loss: 0.0488 - val_accuracy: 0.9876
Epoch 6/10
101/101 [==============================] - 18s 179ms/step - loss: 0.0881 - accuracy: 0.9781 - val_loss: 0.0411 - val_accuracy: 0.9880
Epoch 7/10
101/101 [==============================] - 18s 180ms/step - loss: 0.0767 - accuracy: 0.9779 - val_loss: 0.0542 - val

In [108]:
#
print(n)
models_dg[n].fit_model(image_sets_dg[n],label_sets_dg[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 21s 177ms/step - loss: 0.2062 - accuracy: 0.9559 - val_loss: 0.0429 - val_accuracy: 0.9879
Epoch 2/10
110/110 [==============================] - 19s 176ms/step - loss: 0.1226 - accuracy: 0.9685 - val_loss: 0.0589 - val_accuracy: 0.9845
Epoch 3/10
110/110 [==============================] - 19s 176ms/step - loss: 0.1278 - accuracy: 0.9676 - val_loss: 0.0439 - val_accuracy: 0.9889
Epoch 4/10
110/110 [==============================] - 19s 176ms/step - loss: 0.1116 - accuracy: 0.9742 - val_loss: 0.0472 - val_accuracy: 0.9880
Epoch 5/10
110/110 [==============================] - 19s 175ms/step - loss: 0.1029 - accuracy: 0.9740 - val_loss: 0.0474 - val_accuracy: 0.9876
Epoch 6/10
110/110 [==============================] - 19s 176ms/step - loss: 0.0958 - accuracy: 0.9781 - val_loss: 0.0475 - val_accuracy: 0.9864
Epoch 7/10
110/110 [==============================] - 19s 176ms/step - loss: 0.0899 - accuracy: 0.9776 - val_loss: 0.0715 - val

In [109]:
#new_model_dg_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_dg_e1"
new_model_dg_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_dg_e1"

i=0

for model in models_dg:
    model.save(new_model_dg_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_dg_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model

In [110]:
try:
    del deep_gini_values
    del top_images_by_dg
    del top_labels_by_dg
    del image_sets_dg
    del label_sets_dg
    del models_dg
except:
    print("Error")

In [111]:

gc.collect()

315482

### Softmax 

In [112]:
#softmax values
#se_direction = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/softmax_values.npy"
se_direction = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"

se_values = np.load(se_direction)[len(x_train):] 
print(len(se_values))


6999


In [113]:
# Obtaining top n images by LSA values
top_images_by_se  = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),x_adversary_training)
top_labels_by_se = utils.get_x_of_indexes(list(np.flip(np.argsort(se_values))),y_adversary_training)

In [114]:
m = n_data_points

n = 0
image_sets_se = []
label_sets_se = []

# last
#for i in range(0,len(top_images_by_lsa)//m):

for i in range((len(top_images_by_se)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_se)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_se)%m))
        top_images_by_se_n = np.array(top_images_by_se[:n+m+(len(top_images_by_se)%m)])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m+(len(top_images_by_se)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_se_n = np.array(top_images_by_se[:n+m])
        top_labels_by_se_n = np.array(top_labels_by_se[:n+m])
    image_sets_se.append(top_images_by_se_n)
    label_sets_se.append(top_labels_by_se_n)
    print(len(top_images_by_se_n))
    n += m



0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [115]:
print(model_dir)
models_se = []
for i in range(len(label_sets_se)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_se.append(model)

C:/Users/fjdur/Documents/upc-july/models/models2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [116]:
n=0
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 6s 1s/step - loss: 0.5712 - accuracy: 0.8899 - val_loss: 0.0846 - val_accuracy: 0.9869
Epoch 2/10
6/6 [==============================] - 5s 925ms/step - loss: 0.2853 - accuracy: 0.9318 - val_loss: 0.0751 - val_accuracy: 0.9860
Epoch 3/10
6/6 [==============================] - 5s 922ms/step - loss: 0.1289 - accuracy: 0.9711 - val_loss: 0.0828 - val_accuracy: 0.9783
Epoch 4/10
6/6 [==============================] - 5s 945ms/step - loss: 0.3041 - accuracy: 0.9316 - val_loss: 0.0756 - val_accuracy: 0.9849
Epoch 5/10
6/6 [==============================] - 5s 943ms/step - loss: 0.1661 - accuracy: 0.9413 - val_loss: 0.0986 - val_accuracy: 0.9830
Epoch 6/10
6/6 [==============================] - 5s 945ms/step - loss: 0.1231 - accuracy: 0.9672 - val_loss: 0.1141 - val_accuracy: 0.9831
Epoch 7/10
6/6 [==============================] - 5s 954ms/step - loss: 0.1129 - accuracy: 0.9631 - val_loss: 0.0612 - val_accuracy: 0.9844
Epoch 8/10
6/6 [=====

In [117]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 7s 556ms/step - loss: 0.5420 - accuracy: 0.8986 - val_loss: 0.0705 - val_accuracy: 0.9866
Epoch 2/10
11/11 [==============================] - 5s 529ms/step - loss: 0.2330 - accuracy: 0.9286 - val_loss: 0.1019 - val_accuracy: 0.9841
Epoch 3/10
11/11 [==============================] - 5s 527ms/step - loss: 0.1759 - accuracy: 0.9552 - val_loss: 0.0762 - val_accuracy: 0.9850
Epoch 4/10
11/11 [==============================] - 5s 527ms/step - loss: 0.1351 - accuracy: 0.9690 - val_loss: 0.0663 - val_accuracy: 0.9868
Epoch 5/10
11/11 [==============================] - 5s 528ms/step - loss: 0.1856 - accuracy: 0.9592 - val_loss: 0.0709 - val_accuracy: 0.9865
Epoch 6/10
11/11 [==============================] - 5s 532ms/step - loss: 0.1356 - accuracy: 0.9626 - val_loss: 0.0539 - val_accuracy: 0.9857
Epoch 7/10
11/11 [==============================] - 5s 534ms/step - loss: 0.1434 - accuracy: 0.9663 - val_loss: 0.0605 - val_accuracy: 0.9868
Epoc

In [118]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 8s 411ms/step - loss: 0.5489 - accuracy: 0.8924 - val_loss: 0.0524 - val_accuracy: 0.9845
Epoch 2/10
16/16 [==============================] - 6s 392ms/step - loss: 0.2638 - accuracy: 0.9326 - val_loss: 0.0614 - val_accuracy: 0.9851
Epoch 3/10
16/16 [==============================] - 6s 390ms/step - loss: 0.2333 - accuracy: 0.9471 - val_loss: 0.0553 - val_accuracy: 0.9867
Epoch 4/10
16/16 [==============================] - 6s 393ms/step - loss: 0.2509 - accuracy: 0.9453 - val_loss: 0.0475 - val_accuracy: 0.9889
Epoch 5/10
16/16 [==============================] - 6s 390ms/step - loss: 0.1683 - accuracy: 0.9482 - val_loss: 0.0564 - val_accuracy: 0.9874
Epoch 6/10
16/16 [==============================] - 6s 390ms/step - loss: 0.1214 - accuracy: 0.9586 - val_loss: 0.0456 - val_accuracy: 0.9869
Epoch 7/10
16/16 [==============================] - 6s 390ms/step - loss: 0.1313 - accuracy: 0.9567 - val_loss: 0.0472 - val_accuracy: 0.9888
Epoc

In [119]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 8s 319ms/step - loss: 0.5485 - accuracy: 0.8829 - val_loss: 0.0545 - val_accuracy: 0.9849
Epoch 2/10
22/22 [==============================] - 6s 304ms/step - loss: 0.3243 - accuracy: 0.9174 - val_loss: 0.0592 - val_accuracy: 0.9878
Epoch 3/10
22/22 [==============================] - 6s 304ms/step - loss: 0.2572 - accuracy: 0.9133 - val_loss: 0.0650 - val_accuracy: 0.9860
Epoch 4/10
22/22 [==============================] - 7s 305ms/step - loss: 0.2609 - accuracy: 0.9337 - val_loss: 0.0608 - val_accuracy: 0.9822
Epoch 5/10
22/22 [==============================] - 6s 304ms/step - loss: 0.1912 - accuracy: 0.9421 - val_loss: 0.0490 - val_accuracy: 0.9886
Epoch 6/10
22/22 [==============================] - 6s 304ms/step - loss: 0.1860 - accuracy: 0.9443 - val_loss: 0.0596 - val_accuracy: 0.9854
Epoch 7/10
22/22 [==============================] - 7s 306ms/step - loss: 0.1527 - accuracy: 0.9598 - val_loss: 0.0622 - val_accuracy: 0.9870
Epoc

In [120]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 9s 283ms/step - loss: 0.6076 - accuracy: 0.8775 - val_loss: 0.0496 - val_accuracy: 0.9879
Epoch 2/10
27/27 [==============================] - 7s 270ms/step - loss: 0.3012 - accuracy: 0.9286 - val_loss: 0.0552 - val_accuracy: 0.9864
Epoch 3/10
27/27 [==============================] - 7s 267ms/step - loss: 0.1950 - accuracy: 0.9371 - val_loss: 0.0506 - val_accuracy: 0.9887
Epoch 4/10
27/27 [==============================] - 7s 270ms/step - loss: 0.1938 - accuracy: 0.9418 - val_loss: 0.0740 - val_accuracy: 0.9838
Epoch 5/10
27/27 [==============================] - 7s 266ms/step - loss: 0.2176 - accuracy: 0.9450 - val_loss: 0.0543 - val_accuracy: 0.9878
Epoch 6/10
27/27 [==============================] - 7s 271ms/step - loss: 0.1549 - accuracy: 0.9527 - val_loss: 0.0525 - val_accuracy: 0.9876
Epoch 7/10
27/27 [==============================] - 7s 276ms/step - loss: 0.1584 - accuracy: 0.9510 - val_loss: 0.0487 - val_accuracy: 0.9860
Epoc

In [121]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 9s 250ms/step - loss: 0.4833 - accuracy: 0.9151 - val_loss: 0.0401 - val_accuracy: 0.9892
Epoch 2/10
32/32 [==============================] - 7s 239ms/step - loss: 0.2630 - accuracy: 0.9224 - val_loss: 0.0455 - val_accuracy: 0.9893
Epoch 3/10
32/32 [==============================] - 8s 242ms/step - loss: 0.2801 - accuracy: 0.9247 - val_loss: 0.0533 - val_accuracy: 0.9878
Epoch 4/10
32/32 [==============================] - 8s 240ms/step - loss: 0.1694 - accuracy: 0.9447 - val_loss: 0.0505 - val_accuracy: 0.9877
Epoch 5/10
32/32 [==============================] - 8s 240ms/step - loss: 0.1619 - accuracy: 0.9520 - val_loss: 0.0534 - val_accuracy: 0.9862
Epoch 6/10
32/32 [==============================] - 8s 240ms/step - loss: 0.1694 - accuracy: 0.9475 - val_loss: 0.0460 - val_accuracy: 0.9879
Epoch 7/10
32/32 [==============================] - 8s 240ms/step - loss: 0.1708 - accuracy: 0.9522 - val_loss: 0.0511 - val_accuracy: 0.9875
Epoc

In [122]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 10s 225ms/step - loss: 0.4944 - accuracy: 0.9045 - val_loss: 0.0498 - val_accuracy: 0.9881
Epoch 2/10
38/38 [==============================] - 8s 217ms/step - loss: 0.2804 - accuracy: 0.9349 - val_loss: 0.0574 - val_accuracy: 0.9881
Epoch 3/10
38/38 [==============================] - 8s 217ms/step - loss: 0.2437 - accuracy: 0.9381 - val_loss: 0.0647 - val_accuracy: 0.9826
Epoch 4/10
38/38 [==============================] - 8s 216ms/step - loss: 0.2807 - accuracy: 0.9231 - val_loss: 0.0478 - val_accuracy: 0.9868
Epoch 5/10
38/38 [==============================] - 8s 217ms/step - loss: 0.2013 - accuracy: 0.9473 - val_loss: 0.0465 - val_accuracy: 0.9891
Epoch 6/10
38/38 [==============================] - 8s 217ms/step - loss: 0.2119 - accuracy: 0.9427 - val_loss: 0.0508 - val_accuracy: 0.9876
Epoch 7/10
38/38 [==============================] - 8s 216ms/step - loss: 0.1718 - accuracy: 0.9481 - val_loss: 0.0410 - val_accuracy: 0.9885
Epo

In [123]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 10s 214ms/step - loss: 0.4388 - accuracy: 0.9147 - val_loss: 0.0866 - val_accuracy: 0.9819
Epoch 2/10
43/43 [==============================] - 9s 209ms/step - loss: 0.2408 - accuracy: 0.9383 - val_loss: 0.0379 - val_accuracy: 0.9892
Epoch 3/10
43/43 [==============================] - 9s 212ms/step - loss: 0.2156 - accuracy: 0.9402 - val_loss: 0.0482 - val_accuracy: 0.9874
Epoch 4/10
43/43 [==============================] - 9s 213ms/step - loss: 0.2481 - accuracy: 0.9367 - val_loss: 0.0475 - val_accuracy: 0.9879
Epoch 5/10
43/43 [==============================] - 9s 212ms/step - loss: 0.1872 - accuracy: 0.9460 - val_loss: 0.0389 - val_accuracy: 0.9892
Epoch 6/10
43/43 [==============================] - 9s 213ms/step - loss: 0.1732 - accuracy: 0.9559 - val_loss: 0.0454 - val_accuracy: 0.9883
Epoch 7/10
43/43 [==============================] - 9s 213ms/step - loss: 0.1616 - accuracy: 0.9518 - val_loss: 0.0540 - val_accuracy: 0.9859
Epo

In [124]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 11s 202ms/step - loss: 0.4231 - accuracy: 0.8973 - val_loss: 0.0408 - val_accuracy: 0.9885
Epoch 2/10
48/48 [==============================] - 10s 203ms/step - loss: 0.2901 - accuracy: 0.9361 - val_loss: 0.0429 - val_accuracy: 0.9891
Epoch 3/10
48/48 [==============================] - 10s 205ms/step - loss: 0.2020 - accuracy: 0.9437 - val_loss: 0.0443 - val_accuracy: 0.9887
Epoch 4/10
48/48 [==============================] - 10s 204ms/step - loss: 0.1984 - accuracy: 0.9419 - val_loss: 0.0435 - val_accuracy: 0.9886
Epoch 5/10
48/48 [==============================] - 10s 207ms/step - loss: 0.1472 - accuracy: 0.9554 - val_loss: 0.0398 - val_accuracy: 0.9879
Epoch 6/10
48/48 [==============================] - 10s 205ms/step - loss: 0.1827 - accuracy: 0.9465 - val_loss: 0.0461 - val_accuracy: 0.9879
Epoch 7/10
48/48 [==============================] - 10s 207ms/step - loss: 0.1377 - accuracy: 0.9510 - val_loss: 0.0969 - val_accuracy: 0.98

In [125]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 12s 200ms/step - loss: 0.3599 - accuracy: 0.9226 - val_loss: 0.0566 - val_accuracy: 0.9869
Epoch 2/10
54/54 [==============================] - 11s 202ms/step - loss: 0.2872 - accuracy: 0.9392 - val_loss: 0.0510 - val_accuracy: 0.9874
Epoch 3/10
54/54 [==============================] - 11s 201ms/step - loss: 0.1868 - accuracy: 0.9557 - val_loss: 0.0551 - val_accuracy: 0.9869
Epoch 4/10
54/54 [==============================] - 11s 202ms/step - loss: 0.1954 - accuracy: 0.9511 - val_loss: 0.0460 - val_accuracy: 0.9876
Epoch 5/10
54/54 [==============================] - 11s 203ms/step - loss: 0.1727 - accuracy: 0.9565 - val_loss: 0.0456 - val_accuracy: 0.9893
Epoch 6/10
54/54 [==============================] - 11s 206ms/step - loss: 0.1621 - accuracy: 0.9572 - val_loss: 0.0480 - val_accuracy: 0.9876
Epoch 7/10
54/54 [==============================] - 11s 206ms/step - loss: 0.1291 - accuracy: 0.9625 - val_loss: 0.0741 - val_accuracy: 0.98

In [126]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 13s 199ms/step - loss: 0.3393 - accuracy: 0.9294 - val_loss: 0.0431 - val_accuracy: 0.9884
Epoch 2/10
59/59 [==============================] - 12s 197ms/step - loss: 0.2235 - accuracy: 0.9417 - val_loss: 0.0499 - val_accuracy: 0.9864
Epoch 3/10
59/59 [==============================] - 12s 198ms/step - loss: 0.1878 - accuracy: 0.9473 - val_loss: 0.0610 - val_accuracy: 0.9875
Epoch 4/10
59/59 [==============================] - 12s 198ms/step - loss: 0.1760 - accuracy: 0.9600 - val_loss: 0.0492 - val_accuracy: 0.9864
Epoch 5/10
59/59 [==============================] - 12s 199ms/step - loss: 0.1743 - accuracy: 0.9522 - val_loss: 0.0490 - val_accuracy: 0.9886
Epoch 6/10
59/59 [==============================] - 12s 197ms/step - loss: 0.1330 - accuracy: 0.9623 - val_loss: 0.0537 - val_accuracy: 0.9856
Epoch 7/10
59/59 [==============================] - 12s 200ms/step - loss: 0.1325 - accuracy: 0.9594 - val_loss: 0.0498 - val_accuracy: 0.9

In [127]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 13s 191ms/step - loss: 0.3906 - accuracy: 0.9346 - val_loss: 0.0477 - val_accuracy: 0.9876
Epoch 2/10
64/64 [==============================] - 12s 190ms/step - loss: 0.2123 - accuracy: 0.9488 - val_loss: 0.0492 - val_accuracy: 0.9878
Epoch 3/10
64/64 [==============================] - 12s 190ms/step - loss: 0.1973 - accuracy: 0.9465 - val_loss: 0.0502 - val_accuracy: 0.9874
Epoch 4/10
64/64 [==============================] - 12s 190ms/step - loss: 0.2032 - accuracy: 0.9505 - val_loss: 0.0457 - val_accuracy: 0.9881
Epoch 5/10
64/64 [==============================] - 12s 190ms/step - loss: 0.1402 - accuracy: 0.9627 - val_loss: 0.0474 - val_accuracy: 0.9879
Epoch 6/10
64/64 [==============================] - 12s 191ms/step - loss: 0.1273 - accuracy: 0.9658 - val_loss: 0.0417 - val_accuracy: 0.9883
Epoch 7/10
64/64 [==============================] - 12s 191ms/step - loss: 0.1017 - accuracy: 0.9698 - val_loss: 0.0446 - val_accuracy: 0.9

In [128]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 14s 184ms/step - loss: 0.3298 - accuracy: 0.9331 - val_loss: 0.0841 - val_accuracy: 0.9819
Epoch 2/10
70/70 [==============================] - 13s 183ms/step - loss: 0.1880 - accuracy: 0.9507 - val_loss: 0.0488 - val_accuracy: 0.9873
Epoch 3/10
70/70 [==============================] - 13s 183ms/step - loss: 0.1661 - accuracy: 0.9564 - val_loss: 0.0458 - val_accuracy: 0.9872
Epoch 4/10
70/70 [==============================] - 13s 183ms/step - loss: 0.1601 - accuracy: 0.9573 - val_loss: 0.0536 - val_accuracy: 0.9840
Epoch 5/10
70/70 [==============================] - 13s 183ms/step - loss: 0.1564 - accuracy: 0.9609 - val_loss: 0.0483 - val_accuracy: 0.9879
Epoch 6/10
70/70 [==============================] - 13s 184ms/step - loss: 0.1320 - accuracy: 0.9666 - val_loss: 0.0751 - val_accuracy: 0.9832
Epoch 7/10
70/70 [==============================] - 13s 183ms/step - loss: 0.1264 - accuracy: 0.9669 - val_loss: 0.0572 - val_accuracy: 0.9

In [129]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 15s 178ms/step - loss: 0.3207 - accuracy: 0.9382 - val_loss: 0.0663 - val_accuracy: 0.9846
Epoch 2/10
75/75 [==============================] - 13s 177ms/step - loss: 0.2292 - accuracy: 0.9507 - val_loss: 0.0476 - val_accuracy: 0.9883
Epoch 3/10
75/75 [==============================] - 13s 178ms/step - loss: 0.1454 - accuracy: 0.9638 - val_loss: 0.0476 - val_accuracy: 0.9878
Epoch 4/10
75/75 [==============================] - 14s 183ms/step - loss: 0.1404 - accuracy: 0.9588 - val_loss: 0.0473 - val_accuracy: 0.9882
Epoch 5/10
75/75 [==============================] - 13s 180ms/step - loss: 0.1169 - accuracy: 0.9712 - val_loss: 0.0560 - val_accuracy: 0.9879
Epoch 6/10
75/75 [==============================] - 13s 178ms/step - loss: 0.1321 - accuracy: 0.9684 - val_loss: 0.0560 - val_accuracy: 0.9874
Epoch 7/10
75/75 [==============================] - 13s 178ms/step - loss: 0.1187 - accuracy: 0.9711 - val_loss: 0.0635 - val_accuracy: 0.9

In [130]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 15s 176ms/step - loss: 0.3094 - accuracy: 0.9431 - val_loss: 0.0471 - val_accuracy: 0.9875
Epoch 2/10
80/80 [==============================] - 14s 178ms/step - loss: 0.1618 - accuracy: 0.9597 - val_loss: 0.0480 - val_accuracy: 0.9876
Epoch 3/10
80/80 [==============================] - 14s 175ms/step - loss: 0.1532 - accuracy: 0.9590 - val_loss: 0.0427 - val_accuracy: 0.9887
Epoch 4/10
80/80 [==============================] - 14s 177ms/step - loss: 0.1410 - accuracy: 0.9680 - val_loss: 0.0390 - val_accuracy: 0.9890
Epoch 5/10
80/80 [==============================] - 14s 177ms/step - loss: 0.0970 - accuracy: 0.9749 - val_loss: 0.0479 - val_accuracy: 0.9882
Epoch 6/10
80/80 [==============================] - 14s 176ms/step - loss: 0.1273 - accuracy: 0.9651 - val_loss: 0.0409 - val_accuracy: 0.9879
Epoch 7/10
80/80 [==============================] - 14s 176ms/step - loss: 0.0957 - accuracy: 0.9740 - val_loss: 0.0435 - val_accuracy: 0.9

In [131]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 16s 172ms/step - loss: 0.2366 - accuracy: 0.9491 - val_loss: 0.0395 - val_accuracy: 0.9889
Epoch 2/10
85/85 [==============================] - 15s 173ms/step - loss: 0.1828 - accuracy: 0.9556 - val_loss: 0.0479 - val_accuracy: 0.9883
Epoch 3/10
85/85 [==============================] - 15s 172ms/step - loss: 0.1421 - accuracy: 0.9659 - val_loss: 0.0428 - val_accuracy: 0.9890
Epoch 4/10
85/85 [==============================] - 14s 171ms/step - loss: 0.1468 - accuracy: 0.9642 - val_loss: 0.0483 - val_accuracy: 0.9871
Epoch 5/10
85/85 [==============================] - 15s 171ms/step - loss: 0.1174 - accuracy: 0.9674 - val_loss: 0.0464 - val_accuracy: 0.9878
Epoch 6/10
85/85 [==============================] - 15s 172ms/step - loss: 0.1061 - accuracy: 0.9715 - val_loss: 0.0475 - val_accuracy: 0.9883
Epoch 7/10
85/85 [==============================] - 15s 173ms/step - loss: 0.1044 - accuracy: 0.9753 - val_loss: 0.0458 - val_accuracy: 0.9

In [132]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 19s 188ms/step - loss: 0.2176 - accuracy: 0.9502 - val_loss: 0.0550 - val_accuracy: 0.9871
Epoch 2/10
91/91 [==============================] - 17s 189ms/step - loss: 0.1478 - accuracy: 0.9632 - val_loss: 0.0425 - val_accuracy: 0.9886
Epoch 3/10
91/91 [==============================] - 17s 187ms/step - loss: 0.1238 - accuracy: 0.9676 - val_loss: 0.0615 - val_accuracy: 0.9858
Epoch 4/10
91/91 [==============================] - 17s 186ms/step - loss: 0.1197 - accuracy: 0.9654 - val_loss: 0.0610 - val_accuracy: 0.9853
Epoch 5/10
91/91 [==============================] - 17s 186ms/step - loss: 0.0926 - accuracy: 0.9736 - val_loss: 0.0663 - val_accuracy: 0.9837
Epoch 6/10
91/91 [==============================] - 17s 187ms/step - loss: 0.0804 - accuracy: 0.9757 - val_loss: 0.0537 - val_accuracy: 0.9870
Epoch 7/10
91/91 [==============================] - 17s 188ms/step - loss: 0.0880 - accuracy: 0.9759 - val_loss: 0.0739 - val_accuracy: 0.9

In [133]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 20s 187ms/step - loss: 0.2157 - accuracy: 0.9531 - val_loss: 0.0459 - val_accuracy: 0.9879
Epoch 2/10
96/96 [==============================] - 18s 186ms/step - loss: 0.1686 - accuracy: 0.9576 - val_loss: 0.0425 - val_accuracy: 0.9881
Epoch 3/10
96/96 [==============================] - 18s 186ms/step - loss: 0.1403 - accuracy: 0.9667 - val_loss: 0.0460 - val_accuracy: 0.9883
Epoch 4/10
96/96 [==============================] - 18s 186ms/step - loss: 0.1099 - accuracy: 0.9740 - val_loss: 0.0583 - val_accuracy: 0.9854
Epoch 5/10
96/96 [==============================] - 18s 186ms/step - loss: 0.1001 - accuracy: 0.9699 - val_loss: 0.0512 - val_accuracy: 0.9876
Epoch 6/10
96/96 [==============================] - 18s 186ms/step - loss: 0.1000 - accuracy: 0.9756 - val_loss: 0.0466 - val_accuracy: 0.9871
Epoch 7/10
96/96 [==============================] - 18s 186ms/step - loss: 0.0917 - accuracy: 0.9741 - val_loss: 0.0550 - val_accuracy: 0.9

In [134]:
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 20s 185ms/step - loss: 0.2071 - accuracy: 0.9551 - val_loss: 0.0470 - val_accuracy: 0.9889
Epoch 2/10
101/101 [==============================] - 19s 186ms/step - loss: 0.1615 - accuracy: 0.9611 - val_loss: 0.0607 - val_accuracy: 0.9873
Epoch 3/10
101/101 [==============================] - 19s 184ms/step - loss: 0.1474 - accuracy: 0.9644 - val_loss: 0.0461 - val_accuracy: 0.9870
Epoch 4/10
101/101 [==============================] - 18s 183ms/step - loss: 0.1228 - accuracy: 0.9736 - val_loss: 0.0492 - val_accuracy: 0.9884
Epoch 5/10
101/101 [==============================] - 19s 185ms/step - loss: 0.0915 - accuracy: 0.9776 - val_loss: 0.0477 - val_accuracy: 0.9867
Epoch 6/10
101/101 [==============================] - 19s 185ms/step - loss: 0.0913 - accuracy: 0.9776 - val_loss: 0.0514 - val_accuracy: 0.9854
Epoch 7/10
101/101 [==============================] - 19s 185ms/step - loss: 0.0912 - accuracy: 0.9745 - val_loss: 0.0639 - val

In [135]:
#
print(n)
models_se[n].fit_model(image_sets_se[n],label_sets_se[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 22s 181ms/step - loss: 0.2153 - accuracy: 0.9605 - val_loss: 0.0533 - val_accuracy: 0.9870
Epoch 2/10
110/110 [==============================] - 20s 181ms/step - loss: 0.1502 - accuracy: 0.9662 - val_loss: 0.0506 - val_accuracy: 0.9883
Epoch 3/10
110/110 [==============================] - 20s 182ms/step - loss: 0.1219 - accuracy: 0.9721 - val_loss: 0.0610 - val_accuracy: 0.9860
Epoch 4/10
110/110 [==============================] - 20s 182ms/step - loss: 0.1075 - accuracy: 0.9696 - val_loss: 0.0625 - val_accuracy: 0.9856
Epoch 5/10
110/110 [==============================] - 20s 180ms/step - loss: 0.0941 - accuracy: 0.9743 - val_loss: 0.0513 - val_accuracy: 0.9866
Epoch 6/10
110/110 [==============================] - 20s 181ms/step - loss: 0.1013 - accuracy: 0.9750 - val_loss: 0.0554 - val_accuracy: 0.9871
Epoch 7/10
110/110 [==============================] - 20s 181ms/step - loss: 0.0992 - accuracy: 0.9773 - val_loss: 0.0644 - val

In [136]:
#new_model_se_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_se_e1"
new_model_se_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_se_e1"

i=0

for model in models_se:
    model.save(new_model_se_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_se_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model

In [137]:
try:
    del se_values
    del top_images_by_se
    del top_labels_by_se
    del image_sets_se
    del label_sets_se
    del models_se
except:
    print("Error")

In [138]:
gc.collect()

306302

## Training guided by Random values

In [139]:
#save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/random_values.npy"
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"


random_indexes = np.load(save_dir)[len(x_train):] 
print(len(random_indexes))

6999


In [140]:
len(random_indexes)

6999

In [141]:
# Obtaining top n images by random values
top_images_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),x_adversary_training)
top_labels_by_random = utils.get_x_of_indexes(list(np.flip(np.argsort(random_indexes))),y_adversary_training)

In [142]:
m = n_data_points
n = 0
image_sets_random = []
label_sets_random = []



for i in range((len(top_images_by_random)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_random)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_random)%m))
        top_images_by_random_n = np.array(top_images_by_random[:n+m+(len(top_images_by_random)%m)])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m+(len(top_images_by_random)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_random_n = np.array(top_images_by_random[:n+m])
        top_labels_by_random_n = np.array(top_labels_by_random[:n+m])
    image_sets_random.append(top_images_by_random_n)
    label_sets_random.append(top_labels_by_random_n)
    print(len(top_images_by_random_n))
    n += m




0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [143]:
len(image_sets_random)

20

In [144]:
print(model_dir)

models_random = []
for i in range(len(label_sets_random)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_random.append(model)


C:/Users/fjdur/Documents/upc-july/models/models2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [145]:
n=0
print(n)

models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 8s 1s/step - loss: 0.3897 - accuracy: 0.9469 - val_loss: 0.0733 - val_accuracy: 0.9794
Epoch 2/10
6/6 [==============================] - 6s 1s/step - loss: 0.2330 - accuracy: 0.9637 - val_loss: 0.0589 - val_accuracy: 0.9866
Epoch 3/10
6/6 [==============================] - 6s 1s/step - loss: 0.0516 - accuracy: 0.9877 - val_loss: 0.0544 - val_accuracy: 0.9855
Epoch 4/10
6/6 [==============================] - 6s 1s/step - loss: 0.0787 - accuracy: 0.9702 - val_loss: 0.0613 - val_accuracy: 0.9860
Epoch 5/10
6/6 [==============================] - 6s 1s/step - loss: 0.0599 - accuracy: 0.9806 - val_loss: 0.0805 - val_accuracy: 0.9841
Epoch 6/10
6/6 [==============================] - 6s 1s/step - loss: 0.0578 - accuracy: 0.9919 - val_loss: 0.0622 - val_accuracy: 0.9851
Epoch 7/10
6/6 [==============================] - 6s 1s/step - loss: 0.0314 - accuracy: 0.9888 - val_loss: 0.0820 - val_accuracy: 0.9812
Epoch 8/10
6/6 [=======================

In [146]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 9s 680ms/step - loss: 0.3849 - accuracy: 0.9381 - val_loss: 0.0526 - val_accuracy: 0.9861
Epoch 2/10
11/11 [==============================] - 7s 701ms/step - loss: 0.1453 - accuracy: 0.9617 - val_loss: 0.0738 - val_accuracy: 0.9829
Epoch 3/10
11/11 [==============================] - 7s 702ms/step - loss: 0.1546 - accuracy: 0.9639 - val_loss: 0.0592 - val_accuracy: 0.9866
Epoch 4/10
11/11 [==============================] - 7s 702ms/step - loss: 0.1380 - accuracy: 0.9676 - val_loss: 0.0489 - val_accuracy: 0.9870
Epoch 5/10
11/11 [==============================] - 7s 702ms/step - loss: 0.0815 - accuracy: 0.9769 - val_loss: 0.0558 - val_accuracy: 0.9866
Epoch 6/10
11/11 [==============================] - 7s 697ms/step - loss: 0.1442 - accuracy: 0.9774 - val_loss: 0.0824 - val_accuracy: 0.9824
Epoch 7/10
11/11 [==============================] - 7s 703ms/step - loss: 0.1648 - accuracy: 0.9712 - val_loss: 0.0589 - val_accuracy: 0.9859
Epoc

In [147]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 10s 519ms/step - loss: 0.4269 - accuracy: 0.9443 - val_loss: 0.0807 - val_accuracy: 0.9833
Epoch 2/10
16/16 [==============================] - 8s 514ms/step - loss: 0.1145 - accuracy: 0.9708 - val_loss: 0.0624 - val_accuracy: 0.9836
Epoch 3/10
16/16 [==============================] - 8s 513ms/step - loss: 0.1529 - accuracy: 0.9671 - val_loss: 0.0536 - val_accuracy: 0.9861
Epoch 4/10
16/16 [==============================] - 8s 513ms/step - loss: 0.0858 - accuracy: 0.9773 - val_loss: 0.0538 - val_accuracy: 0.9869
Epoch 5/10
16/16 [==============================] - 8s 519ms/step - loss: 0.1072 - accuracy: 0.9694 - val_loss: 0.0520 - val_accuracy: 0.9871
Epoch 6/10
16/16 [==============================] - 8s 511ms/step - loss: 0.1135 - accuracy: 0.9783 - val_loss: 0.0522 - val_accuracy: 0.9854
Epoch 7/10
16/16 [==============================] - 8s 516ms/step - loss: 0.0805 - accuracy: 0.9776 - val_loss: 0.0790 - val_accuracy: 0.9821
Epo

In [148]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 10s 395ms/step - loss: 0.2572 - accuracy: 0.9409 - val_loss: 0.0650 - val_accuracy: 0.9845
Epoch 2/10
22/22 [==============================] - 9s 403ms/step - loss: 0.1311 - accuracy: 0.9643 - val_loss: 0.0638 - val_accuracy: 0.9865
Epoch 3/10
22/22 [==============================] - 9s 399ms/step - loss: 0.1159 - accuracy: 0.9721 - val_loss: 0.0800 - val_accuracy: 0.9815
Epoch 4/10
22/22 [==============================] - 9s 403ms/step - loss: 0.0984 - accuracy: 0.9807 - val_loss: 0.0479 - val_accuracy: 0.9881
Epoch 5/10
22/22 [==============================] - 9s 400ms/step - loss: 0.0979 - accuracy: 0.9701 - val_loss: 0.0513 - val_accuracy: 0.9871
Epoch 6/10
22/22 [==============================] - 9s 401ms/step - loss: 0.0559 - accuracy: 0.9883 - val_loss: 0.0530 - val_accuracy: 0.9861
Epoch 7/10
22/22 [==============================] - 9s 401ms/step - loss: 0.0907 - accuracy: 0.9767 - val_loss: 0.0431 - val_accuracy: 0.9872
Epo

In [149]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 11s 344ms/step - loss: 0.1948 - accuracy: 0.9578 - val_loss: 0.0531 - val_accuracy: 0.9866
Epoch 2/10
27/27 [==============================] - 9s 355ms/step - loss: 0.1407 - accuracy: 0.9664 - val_loss: 0.0472 - val_accuracy: 0.9871
Epoch 3/10
27/27 [==============================] - 9s 355ms/step - loss: 0.1113 - accuracy: 0.9732 - val_loss: 0.0517 - val_accuracy: 0.9849
Epoch 4/10
27/27 [==============================] - 9s 349ms/step - loss: 0.1041 - accuracy: 0.9770 - val_loss: 0.0637 - val_accuracy: 0.9866
Epoch 5/10
27/27 [==============================] - 9s 350ms/step - loss: 0.0549 - accuracy: 0.9862 - val_loss: 0.0700 - val_accuracy: 0.9833
Epoch 6/10
27/27 [==============================] - 9s 350ms/step - loss: 0.0957 - accuracy: 0.9760 - val_loss: 0.0931 - val_accuracy: 0.9776
Epoch 7/10
27/27 [==============================] - 9s 351ms/step - loss: 0.1056 - accuracy: 0.9756 - val_loss: 0.0589 - val_accuracy: 0.9859
Epo

In [150]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 12s 316ms/step - loss: 0.3498 - accuracy: 0.9457 - val_loss: 0.0397 - val_accuracy: 0.9881
Epoch 2/10
32/32 [==============================] - 10s 322ms/step - loss: 0.1120 - accuracy: 0.9736 - val_loss: 0.0482 - val_accuracy: 0.9884
Epoch 3/10
32/32 [==============================] - 10s 314ms/step - loss: 0.1521 - accuracy: 0.9617 - val_loss: 0.0546 - val_accuracy: 0.9856
Epoch 4/10
32/32 [==============================] - 10s 318ms/step - loss: 0.0945 - accuracy: 0.9797 - val_loss: 0.0538 - val_accuracy: 0.9852
Epoch 5/10
32/32 [==============================] - 10s 314ms/step - loss: 0.0744 - accuracy: 0.9773 - val_loss: 0.0486 - val_accuracy: 0.9866
Epoch 6/10
32/32 [==============================] - 10s 315ms/step - loss: 0.0838 - accuracy: 0.9787 - val_loss: 0.0612 - val_accuracy: 0.9837
Epoch 7/10
32/32 [==============================] - 10s 315ms/step - loss: 0.0962 - accuracy: 0.9740 - val_loss: 0.0511 - val_accuracy: 0.98

In [151]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 12s 280ms/step - loss: 0.2292 - accuracy: 0.9525 - val_loss: 0.0697 - val_accuracy: 0.9834
Epoch 2/10
38/38 [==============================] - 11s 285ms/step - loss: 0.1242 - accuracy: 0.9634 - val_loss: 0.0503 - val_accuracy: 0.9874
Epoch 3/10
38/38 [==============================] - 11s 285ms/step - loss: 0.1232 - accuracy: 0.9753 - val_loss: 0.0526 - val_accuracy: 0.9862
Epoch 4/10
38/38 [==============================] - 11s 288ms/step - loss: 0.1117 - accuracy: 0.9743 - val_loss: 0.0456 - val_accuracy: 0.9874
Epoch 5/10
38/38 [==============================] - 11s 285ms/step - loss: 0.1031 - accuracy: 0.9741 - val_loss: 0.0744 - val_accuracy: 0.9839
Epoch 6/10
38/38 [==============================] - 11s 286ms/step - loss: 0.1105 - accuracy: 0.9800 - val_loss: 0.0518 - val_accuracy: 0.9870
Epoch 7/10
38/38 [==============================] - 11s 284ms/step - loss: 0.0967 - accuracy: 0.9746 - val_loss: 0.0754 - val_accuracy: 0.98

In [152]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 13s 265ms/step - loss: 0.2361 - accuracy: 0.9661 - val_loss: 0.0556 - val_accuracy: 0.9869
Epoch 2/10
43/43 [==============================] - 11s 270ms/step - loss: 0.1459 - accuracy: 0.9631 - val_loss: 0.0546 - val_accuracy: 0.9861
Epoch 3/10
43/43 [==============================] - 11s 269ms/step - loss: 0.1329 - accuracy: 0.9673 - val_loss: 0.0540 - val_accuracy: 0.9869
Epoch 4/10
43/43 [==============================] - 12s 280ms/step - loss: 0.1143 - accuracy: 0.9677 - val_loss: 0.0739 - val_accuracy: 0.9800
Epoch 5/10
43/43 [==============================] - 12s 272ms/step - loss: 0.1204 - accuracy: 0.9657 - val_loss: 0.0455 - val_accuracy: 0.9871
Epoch 6/10
43/43 [==============================] - 12s 271ms/step - loss: 0.1018 - accuracy: 0.9720 - val_loss: 0.0734 - val_accuracy: 0.9859
Epoch 7/10
43/43 [==============================] - 11s 271ms/step - loss: 0.1107 - accuracy: 0.9784 - val_loss: 0.0596 - val_accuracy: 0.98

In [153]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 14s 256ms/step - loss: 0.1935 - accuracy: 0.9673 - val_loss: 0.0450 - val_accuracy: 0.9869
Epoch 2/10
48/48 [==============================] - 13s 275ms/step - loss: 0.1406 - accuracy: 0.9666 - val_loss: 0.0531 - val_accuracy: 0.9864
Epoch 3/10
48/48 [==============================] - 13s 265ms/step - loss: 0.1284 - accuracy: 0.9737 - val_loss: 0.0528 - val_accuracy: 0.9874
Epoch 4/10
48/48 [==============================] - 12s 259ms/step - loss: 0.1025 - accuracy: 0.9758 - val_loss: 0.0583 - val_accuracy: 0.9848
Epoch 5/10
48/48 [==============================] - 12s 253ms/step - loss: 0.0944 - accuracy: 0.9759 - val_loss: 0.0621 - val_accuracy: 0.9861
Epoch 6/10
48/48 [==============================] - 12s 253ms/step - loss: 0.0863 - accuracy: 0.9753 - val_loss: 0.0574 - val_accuracy: 0.9854
Epoch 7/10
48/48 [==============================] - 12s 256ms/step - loss: 0.1150 - accuracy: 0.9758 - val_loss: 0.0555 - val_accuracy: 0.98

In [154]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 14s 234ms/step - loss: 0.2379 - accuracy: 0.9504 - val_loss: 0.0622 - val_accuracy: 0.9869
Epoch 2/10
54/54 [==============================] - 12s 233ms/step - loss: 0.1289 - accuracy: 0.9710 - val_loss: 0.0399 - val_accuracy: 0.9894
Epoch 3/10
54/54 [==============================] - 12s 232ms/step - loss: 0.1005 - accuracy: 0.9778 - val_loss: 0.0463 - val_accuracy: 0.9881
Epoch 4/10
54/54 [==============================] - 13s 236ms/step - loss: 0.0788 - accuracy: 0.9800 - val_loss: 0.0563 - val_accuracy: 0.9852
Epoch 5/10
54/54 [==============================] - 12s 233ms/step - loss: 0.0932 - accuracy: 0.9722 - val_loss: 0.0453 - val_accuracy: 0.9888
Epoch 6/10
54/54 [==============================] - 12s 232ms/step - loss: 0.0774 - accuracy: 0.9827 - val_loss: 0.0468 - val_accuracy: 0.9885
Epoch 7/10
54/54 [==============================] - 12s 232ms/step - loss: 0.0848 - accuracy: 0.9753 - val_loss: 0.0583 - val_accuracy: 0.98

In [155]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 15s 226ms/step - loss: 0.2191 - accuracy: 0.9620 - val_loss: 0.0448 - val_accuracy: 0.9881
Epoch 2/10
59/59 [==============================] - 13s 230ms/step - loss: 0.1301 - accuracy: 0.9702 - val_loss: 0.0499 - val_accuracy: 0.9876
Epoch 3/10
59/59 [==============================] - 14s 232ms/step - loss: 0.0994 - accuracy: 0.9722 - val_loss: 0.0489 - val_accuracy: 0.9876
Epoch 4/10
59/59 [==============================] - 14s 231ms/step - loss: 0.1090 - accuracy: 0.9745 - val_loss: 0.0630 - val_accuracy: 0.9849
Epoch 5/10
59/59 [==============================] - 13s 229ms/step - loss: 0.0879 - accuracy: 0.9786 - val_loss: 0.0682 - val_accuracy: 0.9871
Epoch 6/10
59/59 [==============================] - 13s 229ms/step - loss: 0.0884 - accuracy: 0.9785 - val_loss: 0.0572 - val_accuracy: 0.9870
Epoch 7/10
59/59 [==============================] - 13s 229ms/step - loss: 0.0737 - accuracy: 0.9815 - val_loss: 0.0483 - val_accuracy: 0.9

In [156]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 16s 222ms/step - loss: 0.2817 - accuracy: 0.9506 - val_loss: 0.0630 - val_accuracy: 0.9848
Epoch 2/10
64/64 [==============================] - 14s 225ms/step - loss: 0.1228 - accuracy: 0.9724 - val_loss: 0.0541 - val_accuracy: 0.9867
Epoch 3/10
64/64 [==============================] - 14s 221ms/step - loss: 0.1265 - accuracy: 0.9706 - val_loss: 0.0496 - val_accuracy: 0.9875
Epoch 4/10
64/64 [==============================] - 14s 222ms/step - loss: 0.1122 - accuracy: 0.9723 - val_loss: 0.0537 - val_accuracy: 0.9876
Epoch 5/10
64/64 [==============================] - 14s 222ms/step - loss: 0.0915 - accuracy: 0.9743 - val_loss: 0.0415 - val_accuracy: 0.9879
Epoch 6/10
64/64 [==============================] - 15s 236ms/step - loss: 0.0838 - accuracy: 0.9810 - val_loss: 0.0596 - val_accuracy: 0.9831
Epoch 7/10
64/64 [==============================] - 14s 217ms/step - loss: 0.0780 - accuracy: 0.9844 - val_loss: 0.0583 - val_accuracy: 0.9

In [157]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 15s 194ms/step - loss: 0.2608 - accuracy: 0.9532 - val_loss: 0.0420 - val_accuracy: 0.9891
Epoch 2/10
70/70 [==============================] - 14s 195ms/step - loss: 0.1397 - accuracy: 0.9649 - val_loss: 0.0574 - val_accuracy: 0.9879
Epoch 3/10
70/70 [==============================] - 13s 193ms/step - loss: 0.1313 - accuracy: 0.9669 - val_loss: 0.0652 - val_accuracy: 0.9869
Epoch 4/10
70/70 [==============================] - 13s 194ms/step - loss: 0.1061 - accuracy: 0.9716 - val_loss: 0.0509 - val_accuracy: 0.9886
Epoch 5/10
70/70 [==============================] - 13s 193ms/step - loss: 0.0960 - accuracy: 0.9737 - val_loss: 0.0483 - val_accuracy: 0.9869
Epoch 6/10
70/70 [==============================] - 14s 195ms/step - loss: 0.0606 - accuracy: 0.9871 - val_loss: 0.0704 - val_accuracy: 0.9817
Epoch 7/10
70/70 [==============================] - 13s 194ms/step - loss: 0.0885 - accuracy: 0.9812 - val_loss: 0.1628 - val_accuracy: 0.9

In [158]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 15s 189ms/step - loss: 0.1845 - accuracy: 0.9668 - val_loss: 0.0549 - val_accuracy: 0.9875
Epoch 2/10
75/75 [==============================] - 14s 188ms/step - loss: 0.1174 - accuracy: 0.9691 - val_loss: 0.0549 - val_accuracy: 0.9876
Epoch 3/10
75/75 [==============================] - 14s 188ms/step - loss: 0.1195 - accuracy: 0.9702 - val_loss: 0.0605 - val_accuracy: 0.9840
Epoch 4/10
75/75 [==============================] - 14s 188ms/step - loss: 0.0905 - accuracy: 0.9740 - val_loss: 0.0542 - val_accuracy: 0.9862
Epoch 5/10
75/75 [==============================] - 14s 188ms/step - loss: 0.0928 - accuracy: 0.9761 - val_loss: 0.0537 - val_accuracy: 0.9869
Epoch 6/10
75/75 [==============================] - 14s 192ms/step - loss: 0.0919 - accuracy: 0.9755 - val_loss: 0.0534 - val_accuracy: 0.9871
Epoch 7/10
75/75 [==============================] - 14s 188ms/step - loss: 0.0793 - accuracy: 0.9792 - val_loss: 0.0456 - val_accuracy: 0.9

In [159]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 16s 186ms/step - loss: 0.2243 - accuracy: 0.9603 - val_loss: 0.0500 - val_accuracy: 0.9864
Epoch 2/10
80/80 [==============================] - 15s 187ms/step - loss: 0.1155 - accuracy: 0.9718 - val_loss: 0.0592 - val_accuracy: 0.9862
Epoch 3/10
80/80 [==============================] - 15s 185ms/step - loss: 0.1299 - accuracy: 0.9722 - val_loss: 0.0436 - val_accuracy: 0.9891
Epoch 4/10
80/80 [==============================] - 15s 189ms/step - loss: 0.0862 - accuracy: 0.9750 - val_loss: 0.0504 - val_accuracy: 0.9875
Epoch 5/10
80/80 [==============================] - 15s 185ms/step - loss: 0.0952 - accuracy: 0.9805 - val_loss: 0.0484 - val_accuracy: 0.9869
Epoch 6/10
80/80 [==============================] - 15s 186ms/step - loss: 0.1077 - accuracy: 0.9714 - val_loss: 0.0559 - val_accuracy: 0.9877
Epoch 7/10
80/80 [==============================] - 15s 185ms/step - loss: 0.0779 - accuracy: 0.9802 - val_loss: 0.0612 - val_accuracy: 0.9

In [160]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 17s 180ms/step - loss: 0.2055 - accuracy: 0.9585 - val_loss: 0.0647 - val_accuracy: 0.9829
Epoch 2/10
85/85 [==============================] - 15s 179ms/step - loss: 0.1514 - accuracy: 0.9672 - val_loss: 0.0510 - val_accuracy: 0.9878
Epoch 3/10
85/85 [==============================] - 15s 180ms/step - loss: 0.0965 - accuracy: 0.9764 - val_loss: 0.0525 - val_accuracy: 0.9870
Epoch 4/10
85/85 [==============================] - 15s 181ms/step - loss: 0.0997 - accuracy: 0.9745 - val_loss: 0.0476 - val_accuracy: 0.9879
Epoch 5/10
85/85 [==============================] - 15s 179ms/step - loss: 0.1067 - accuracy: 0.9766 - val_loss: 0.0550 - val_accuracy: 0.9877
Epoch 6/10
85/85 [==============================] - 15s 180ms/step - loss: 0.0906 - accuracy: 0.9753 - val_loss: 0.0557 - val_accuracy: 0.9860
Epoch 7/10
85/85 [==============================] - 15s 180ms/step - loss: 0.1075 - accuracy: 0.9753 - val_loss: 0.0684 - val_accuracy: 0.9

In [161]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 17s 177ms/step - loss: 0.2403 - accuracy: 0.9508 - val_loss: 0.0622 - val_accuracy: 0.9855
Epoch 2/10
91/91 [==============================] - 16s 180ms/step - loss: 0.1123 - accuracy: 0.9730 - val_loss: 0.0441 - val_accuracy: 0.9889
Epoch 3/10
91/91 [==============================] - 16s 177ms/step - loss: 0.1485 - accuracy: 0.9635 - val_loss: 0.0464 - val_accuracy: 0.9880
Epoch 4/10
91/91 [==============================] - 16s 177ms/step - loss: 0.1058 - accuracy: 0.9705 - val_loss: 0.0432 - val_accuracy: 0.9881
Epoch 5/10
91/91 [==============================] - 16s 177ms/step - loss: 0.1059 - accuracy: 0.9738 - val_loss: 0.0505 - val_accuracy: 0.9872
Epoch 6/10
91/91 [==============================] - 16s 179ms/step - loss: 0.0897 - accuracy: 0.9764 - val_loss: 0.0465 - val_accuracy: 0.9879
Epoch 7/10
91/91 [==============================] - 16s 178ms/step - loss: 0.0698 - accuracy: 0.9802 - val_loss: 0.0561 - val_accuracy: 0.9

In [162]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 18s 176ms/step - loss: 0.1925 - accuracy: 0.9601 - val_loss: 0.0443 - val_accuracy: 0.9879
Epoch 2/10
96/96 [==============================] - 17s 176ms/step - loss: 0.1526 - accuracy: 0.9667 - val_loss: 0.0647 - val_accuracy: 0.9861
Epoch 3/10
96/96 [==============================] - 17s 176ms/step - loss: 0.1149 - accuracy: 0.9735 - val_loss: 0.0629 - val_accuracy: 0.9861
Epoch 4/10
96/96 [==============================] - 17s 176ms/step - loss: 0.1197 - accuracy: 0.9726 - val_loss: 0.0500 - val_accuracy: 0.9871
Epoch 5/10
96/96 [==============================] - 17s 175ms/step - loss: 0.0950 - accuracy: 0.9762 - val_loss: 0.0658 - val_accuracy: 0.9826
Epoch 6/10
96/96 [==============================] - 17s 177ms/step - loss: 0.0931 - accuracy: 0.9776 - val_loss: 0.0500 - val_accuracy: 0.9876
Epoch 7/10
96/96 [==============================] - 17s 175ms/step - loss: 0.0868 - accuracy: 0.9805 - val_loss: 0.0966 - val_accuracy: 0.9

In [163]:
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 19s 172ms/step - loss: 0.2029 - accuracy: 0.9560 - val_loss: 0.0417 - val_accuracy: 0.9880
Epoch 2/10
101/101 [==============================] - 17s 174ms/step - loss: 0.1471 - accuracy: 0.9649 - val_loss: 0.0770 - val_accuracy: 0.9835
Epoch 3/10
101/101 [==============================] - 18s 174ms/step - loss: 0.1391 - accuracy: 0.9694 - val_loss: 0.0461 - val_accuracy: 0.9880
Epoch 4/10
101/101 [==============================] - 17s 174ms/step - loss: 0.0924 - accuracy: 0.9732 - val_loss: 0.0535 - val_accuracy: 0.9864
Epoch 5/10
101/101 [==============================] - 17s 173ms/step - loss: 0.1028 - accuracy: 0.9725 - val_loss: 0.0451 - val_accuracy: 0.9884
Epoch 6/10
101/101 [==============================] - 17s 173ms/step - loss: 0.1032 - accuracy: 0.9740 - val_loss: 0.0553 - val_accuracy: 0.9851
Epoch 7/10
101/101 [==============================] - 17s 173ms/step - loss: 0.0857 - accuracy: 0.9774 - val_loss: 0.0467 - val

In [164]:
#
print(n)
models_random[n].fit_model(image_sets_random[n],label_sets_random[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 20s 170ms/step - loss: 0.1963 - accuracy: 0.9624 - val_loss: 0.0518 - val_accuracy: 0.9865
Epoch 2/10
110/110 [==============================] - 19s 171ms/step - loss: 0.1444 - accuracy: 0.9663 - val_loss: 0.0471 - val_accuracy: 0.9870
Epoch 3/10
110/110 [==============================] - 19s 171ms/step - loss: 0.1335 - accuracy: 0.9658 - val_loss: 0.0465 - val_accuracy: 0.9876
Epoch 4/10
110/110 [==============================] - 19s 170ms/step - loss: 0.1149 - accuracy: 0.9684 - val_loss: 0.1193 - val_accuracy: 0.9681
Epoch 5/10
110/110 [==============================] - 19s 170ms/step - loss: 0.1198 - accuracy: 0.9714 - val_loss: 0.0491 - val_accuracy: 0.9873
Epoch 6/10
110/110 [==============================] - 19s 170ms/step - loss: 0.0892 - accuracy: 0.9767 - val_loss: 0.0509 - val_accuracy: 0.9872
Epoch 7/10
110/110 [==============================] - 19s 173ms/step - loss: 0.0776 - accuracy: 0.9792 - val_loss: 0.0507 - val

In [165]:
#new_model_random_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C2/"+dataset+"_model_c2_aug_random_e1"
new_model_random_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_random_e1"

i=0

for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_random_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/mo

In [166]:
try:
    del random_indexes
    del top_images_by_random
    del top_labels_by_random
    del image_sets_random
    del label_sets_random
    del models_random
except:
    print("Error")

In [167]:
import gc

In [168]:
gc.collect()

274003

## Training guided by NC

In [169]:
# NC
nc_values = []
for i in range(1,20):
    #save_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/NC_values/nc_values_"+str(i)+".npy"
    #save_dir = "C:/Users/fjdurlop/Documents/upc/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"
    save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/"+dataset+"_nc_values_"+str(i)+".npy"

    #print(save_dir_rand)
    tmp_values = np.load(save_dir)
    #print(tmp_values.shape)
    nc_values = np.append(nc_values,tmp_values)
    
nc_values = nc_values[len(x_train):] 
print(len(nc_values))

6999


In [170]:
nc_values.shape

(6999,)

In [171]:

top_images_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),x_adversary_training)
top_labels_by_nc = utils.get_x_of_indexes(list(np.flip(np.argsort(nc_values))),y_adversary_training)


In [172]:
m = n_data_points
n = 0
image_sets_nc = []
label_sets_nc = []


for i in range((len(top_images_by_nc)//m)):
    print(i,":")
    if (i+1 >= ((len(top_images_by_nc)//m))):
        print("Last")
        print(0," -> ",n+m+(len(top_images_by_nc)%m))
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m+(len(top_images_by_nc)%m)])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m+(len(top_images_by_nc)%m)])
    else:
        print(0," -> ",m+n)
        top_images_by_nc_n = np.array(top_images_by_nc[:n+m])
        top_labels_by_nc_n = np.array(top_labels_by_nc[:n+m])
    image_sets_nc.append(top_images_by_nc_n)
    label_sets_nc.append(top_labels_by_nc_n)
    print(len(top_images_by_nc_n))
    n += m




0 :
0  ->  340
340
1 :
0  ->  680
680
2 :
0  ->  1020
1020
3 :
0  ->  1360
1360
4 :
0  ->  1700
1700
5 :
0  ->  2040
2040
6 :
0  ->  2380
2380
7 :
0  ->  2720
2720
8 :
0  ->  3060
3060
9 :
0  ->  3400
3400
10 :
0  ->  3740
3740
11 :
0  ->  4080
4080
12 :
0  ->  4420
4420
13 :
0  ->  4760
4760
14 :
0  ->  5100
5100
15 :
0  ->  5440
5440
16 :
0  ->  5780
5780
17 :
0  ->  6120
6120
18 :
0  ->  6460
6460
19 :
Last
0  ->  6999
6999


In [173]:
print(model_dir)

models_nc = []
for i in range(len(label_sets_nc)):
    print(i,":")
    model = utils.My_model(dataset,True,model_dir)
    model.compile_model()
    models_nc.append(model)


C:/Users/fjdur/Documents/upc-july/models/models2
0 :
Model loaded correctly
Model compiled
1 :
Model loaded correctly
Model compiled
2 :
Model loaded correctly
Model compiled
3 :
Model loaded correctly
Model compiled
4 :
Model loaded correctly
Model compiled
5 :
Model loaded correctly
Model compiled
6 :
Model loaded correctly
Model compiled
7 :
Model loaded correctly
Model compiled
8 :
Model loaded correctly
Model compiled
9 :
Model loaded correctly
Model compiled
10 :
Model loaded correctly
Model compiled
11 :
Model loaded correctly
Model compiled
12 :
Model loaded correctly
Model compiled
13 :
Model loaded correctly
Model compiled
14 :
Model loaded correctly
Model compiled
15 :
Model loaded correctly
Model compiled
16 :
Model loaded correctly
Model compiled
17 :
Model loaded correctly
Model compiled
18 :
Model loaded correctly
Model compiled
19 :
Model loaded correctly
Model compiled


In [174]:
n=0
print(n)

models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

0
Epoch 1/10
6/6 [==============================] - 6s 986ms/step - loss: 0.2479 - accuracy: 0.9529 - val_loss: 0.0940 - val_accuracy: 0.9789
Epoch 2/10
6/6 [==============================] - 5s 933ms/step - loss: 0.1265 - accuracy: 0.9734 - val_loss: 0.0566 - val_accuracy: 0.9838
Epoch 3/10
6/6 [==============================] - 5s 936ms/step - loss: 0.1456 - accuracy: 0.9700 - val_loss: 0.0566 - val_accuracy: 0.9836
Epoch 4/10
6/6 [==============================] - 5s 938ms/step - loss: 0.0859 - accuracy: 0.9825 - val_loss: 0.3216 - val_accuracy: 0.9629
Epoch 5/10
6/6 [==============================] - 5s 941ms/step - loss: 0.0829 - accuracy: 0.9807 - val_loss: 0.0533 - val_accuracy: 0.9849
Epoch 6/10
6/6 [==============================] - 5s 949ms/step - loss: 0.0195 - accuracy: 0.9966 - val_loss: 0.0776 - val_accuracy: 0.9831
Epoch 7/10
6/6 [==============================] - 5s 960ms/step - loss: 0.0201 - accuracy: 0.9965 - val_loss: 0.1091 - val_accuracy: 0.9771
Epoch 8/10
6/6 [==

In [175]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

1
Epoch 1/10
11/11 [==============================] - 7s 578ms/step - loss: 0.2215 - accuracy: 0.9462 - val_loss: 0.0453 - val_accuracy: 0.9873
Epoch 2/10
11/11 [==============================] - 6s 547ms/step - loss: 0.0944 - accuracy: 0.9736 - val_loss: 0.0524 - val_accuracy: 0.9861
Epoch 3/10
11/11 [==============================] - 6s 543ms/step - loss: 0.1034 - accuracy: 0.9759 - val_loss: 0.0516 - val_accuracy: 0.9854
Epoch 4/10
11/11 [==============================] - 6s 549ms/step - loss: 0.0468 - accuracy: 0.9825 - val_loss: 0.0535 - val_accuracy: 0.9854
Epoch 5/10
11/11 [==============================] - 6s 551ms/step - loss: 0.0318 - accuracy: 0.9909 - val_loss: 0.1035 - val_accuracy: 0.9837
Epoch 6/10
11/11 [==============================] - 6s 546ms/step - loss: 0.0728 - accuracy: 0.9833 - val_loss: 0.0602 - val_accuracy: 0.9843
Epoch 7/10
11/11 [==============================] - 6s 548ms/step - loss: 0.0417 - accuracy: 0.9919 - val_loss: 0.0658 - val_accuracy: 0.9837
Epoc

In [176]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

2
Epoch 1/10
16/16 [==============================] - 8s 451ms/step - loss: 0.2269 - accuracy: 0.9594 - val_loss: 0.0449 - val_accuracy: 0.9871
Epoch 2/10
16/16 [==============================] - 6s 424ms/step - loss: 0.0927 - accuracy: 0.9760 - val_loss: 0.0835 - val_accuracy: 0.9831
Epoch 3/10
16/16 [==============================] - 7s 427ms/step - loss: 0.0608 - accuracy: 0.9860 - val_loss: 0.0577 - val_accuracy: 0.9857
Epoch 4/10
16/16 [==============================] - 6s 427ms/step - loss: 0.0719 - accuracy: 0.9829 - val_loss: 0.0534 - val_accuracy: 0.9854
Epoch 5/10
16/16 [==============================] - 7s 429ms/step - loss: 0.0337 - accuracy: 0.9922 - val_loss: 0.0889 - val_accuracy: 0.9822
Epoch 6/10
16/16 [==============================] - 6s 426ms/step - loss: 0.0987 - accuracy: 0.9778 - val_loss: 0.0529 - val_accuracy: 0.9858
Epoch 7/10
16/16 [==============================] - 7s 428ms/step - loss: 0.0370 - accuracy: 0.9838 - val_loss: 0.2426 - val_accuracy: 0.9779
Epoc

In [177]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

3
Epoch 1/10
22/22 [==============================] - 9s 355ms/step - loss: 0.1328 - accuracy: 0.9654 - val_loss: 0.0456 - val_accuracy: 0.9881
Epoch 2/10
22/22 [==============================] - 7s 340ms/step - loss: 0.0799 - accuracy: 0.9703 - val_loss: 0.0593 - val_accuracy: 0.9872
Epoch 3/10
22/22 [==============================] - 7s 341ms/step - loss: 0.0876 - accuracy: 0.9797 - val_loss: 0.0577 - val_accuracy: 0.9849
Epoch 4/10
22/22 [==============================] - 7s 338ms/step - loss: 0.0718 - accuracy: 0.9763 - val_loss: 0.0912 - val_accuracy: 0.9797
Epoch 5/10
22/22 [==============================] - 7s 337ms/step - loss: 0.0804 - accuracy: 0.9817 - val_loss: 0.0503 - val_accuracy: 0.9878
Epoch 6/10
22/22 [==============================] - 7s 341ms/step - loss: 0.0432 - accuracy: 0.9836 - val_loss: 0.0524 - val_accuracy: 0.9863
Epoch 7/10
22/22 [==============================] - 7s 341ms/step - loss: 0.0405 - accuracy: 0.9917 - val_loss: 0.0668 - val_accuracy: 0.9823
Epoc

In [178]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

4
Epoch 1/10
27/27 [==============================] - 10s 311ms/step - loss: 0.1649 - accuracy: 0.9667 - val_loss: 0.0483 - val_accuracy: 0.9866
Epoch 2/10
27/27 [==============================] - 8s 301ms/step - loss: 0.0989 - accuracy: 0.9772 - val_loss: 0.0574 - val_accuracy: 0.9855
Epoch 3/10
27/27 [==============================] - 8s 302ms/step - loss: 0.0990 - accuracy: 0.9734 - val_loss: 0.0526 - val_accuracy: 0.9886
Epoch 4/10
27/27 [==============================] - 8s 303ms/step - loss: 0.0750 - accuracy: 0.9850 - val_loss: 0.0640 - val_accuracy: 0.9827
Epoch 5/10
27/27 [==============================] - 8s 309ms/step - loss: 0.0431 - accuracy: 0.9859 - val_loss: 0.0717 - val_accuracy: 0.9828
Epoch 6/10
27/27 [==============================] - 8s 299ms/step - loss: 0.0485 - accuracy: 0.9864 - val_loss: 0.1119 - val_accuracy: 0.9799
Epoch 7/10
27/27 [==============================] - 8s 299ms/step - loss: 0.0657 - accuracy: 0.9807 - val_loss: 0.3208 - val_accuracy: 0.9622
Epo

In [179]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

5
Epoch 1/10
32/32 [==============================] - 10s 278ms/step - loss: 0.1173 - accuracy: 0.9714 - val_loss: 0.1281 - val_accuracy: 0.9823
Epoch 2/10
32/32 [==============================] - 8s 270ms/step - loss: 0.0849 - accuracy: 0.9758 - val_loss: 0.0721 - val_accuracy: 0.9854
Epoch 3/10
32/32 [==============================] - 8s 269ms/step - loss: 0.0726 - accuracy: 0.9817 - val_loss: 0.0639 - val_accuracy: 0.9867
Epoch 4/10
32/32 [==============================] - 8s 270ms/step - loss: 0.0934 - accuracy: 0.9779 - val_loss: 0.0789 - val_accuracy: 0.9836
Epoch 5/10
32/32 [==============================] - 8s 270ms/step - loss: 0.0711 - accuracy: 0.9805 - val_loss: 0.0527 - val_accuracy: 0.9861
Epoch 6/10
32/32 [==============================] - 9s 271ms/step - loss: 0.0619 - accuracy: 0.9834 - val_loss: 0.0596 - val_accuracy: 0.9876
Epoch 7/10
32/32 [==============================] - 9s 273ms/step - loss: 0.0623 - accuracy: 0.9824 - val_loss: 0.0732 - val_accuracy: 0.9819
Epo

In [180]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

6
Epoch 1/10
38/38 [==============================] - 11s 250ms/step - loss: 0.2211 - accuracy: 0.9610 - val_loss: 0.1981 - val_accuracy: 0.9791
Epoch 2/10
38/38 [==============================] - 9s 243ms/step - loss: 0.1282 - accuracy: 0.9695 - val_loss: 0.0561 - val_accuracy: 0.9851
Epoch 3/10
38/38 [==============================] - 9s 250ms/step - loss: 0.0984 - accuracy: 0.9817 - val_loss: 0.0507 - val_accuracy: 0.9882
Epoch 4/10
38/38 [==============================] - 9s 248ms/step - loss: 0.0824 - accuracy: 0.9771 - val_loss: 0.0447 - val_accuracy: 0.9871
Epoch 5/10
38/38 [==============================] - 9s 250ms/step - loss: 0.1076 - accuracy: 0.9717 - val_loss: 0.0690 - val_accuracy: 0.9850
Epoch 6/10
38/38 [==============================] - 10s 254ms/step - loss: 0.0751 - accuracy: 0.9754 - val_loss: 0.0777 - val_accuracy: 0.9789
Epoch 7/10
38/38 [==============================] - 9s 251ms/step - loss: 0.0680 - accuracy: 0.9796 - val_loss: 0.0779 - val_accuracy: 0.9814
Ep

In [181]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

7
Epoch 1/10
43/43 [==============================] - 11s 237ms/step - loss: 0.1757 - accuracy: 0.9619 - val_loss: 0.0528 - val_accuracy: 0.9859
Epoch 2/10
43/43 [==============================] - 10s 243ms/step - loss: 0.1070 - accuracy: 0.9755 - val_loss: 0.0579 - val_accuracy: 0.9837
Epoch 3/10
43/43 [==============================] - 10s 242ms/step - loss: 0.1051 - accuracy: 0.9706 - val_loss: 0.0520 - val_accuracy: 0.9857
Epoch 4/10
43/43 [==============================] - 10s 242ms/step - loss: 0.0870 - accuracy: 0.9849 - val_loss: 0.0537 - val_accuracy: 0.9859
Epoch 5/10
43/43 [==============================] - 10s 244ms/step - loss: 0.1009 - accuracy: 0.9759 - val_loss: 0.0487 - val_accuracy: 0.9874
Epoch 6/10
43/43 [==============================] - 10s 245ms/step - loss: 0.0888 - accuracy: 0.9797 - val_loss: 0.0486 - val_accuracy: 0.9868
Epoch 7/10
43/43 [==============================] - 10s 245ms/step - loss: 0.0826 - accuracy: 0.9846 - val_loss: 0.0596 - val_accuracy: 0.98

In [182]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

8
Epoch 1/10
48/48 [==============================] - 13s 238ms/step - loss: 0.1489 - accuracy: 0.9703 - val_loss: 0.0468 - val_accuracy: 0.9868
Epoch 2/10
48/48 [==============================] - 11s 242ms/step - loss: 0.1119 - accuracy: 0.9730 - val_loss: 0.0405 - val_accuracy: 0.9885
Epoch 3/10
48/48 [==============================] - 12s 244ms/step - loss: 0.0921 - accuracy: 0.9756 - val_loss: 0.0412 - val_accuracy: 0.9884
Epoch 4/10
48/48 [==============================] - 11s 241ms/step - loss: 0.0835 - accuracy: 0.9779 - val_loss: 0.0573 - val_accuracy: 0.9861
Epoch 5/10
48/48 [==============================] - 11s 241ms/step - loss: 0.0870 - accuracy: 0.9784 - val_loss: 0.0449 - val_accuracy: 0.9871
Epoch 6/10
48/48 [==============================] - 11s 241ms/step - loss: 0.0760 - accuracy: 0.9791 - val_loss: 0.0499 - val_accuracy: 0.9871
Epoch 7/10
48/48 [==============================] - 11s 241ms/step - loss: 0.0643 - accuracy: 0.9830 - val_loss: 0.0644 - val_accuracy: 0.98

In [183]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

9
Epoch 1/10
54/54 [==============================] - 14s 230ms/step - loss: 0.1896 - accuracy: 0.9670 - val_loss: 0.0402 - val_accuracy: 0.9891
Epoch 2/10
54/54 [==============================] - 12s 227ms/step - loss: 0.1273 - accuracy: 0.9657 - val_loss: 0.0431 - val_accuracy: 0.9884
Epoch 3/10
54/54 [==============================] - 12s 227ms/step - loss: 0.0892 - accuracy: 0.9734 - val_loss: 0.0599 - val_accuracy: 0.9855
Epoch 4/10
54/54 [==============================] - 12s 226ms/step - loss: 0.0790 - accuracy: 0.9788 - val_loss: 0.0472 - val_accuracy: 0.9867
Epoch 5/10
54/54 [==============================] - 12s 227ms/step - loss: 0.0877 - accuracy: 0.9756 - val_loss: 0.0599 - val_accuracy: 0.9865
Epoch 6/10
54/54 [==============================] - 12s 227ms/step - loss: 0.0754 - accuracy: 0.9794 - val_loss: 0.0717 - val_accuracy: 0.9824
Epoch 7/10
54/54 [==============================] - 12s 230ms/step - loss: 0.0921 - accuracy: 0.9737 - val_loss: 0.4001 - val_accuracy: 0.97

In [184]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

10
Epoch 1/10
59/59 [==============================] - 14s 220ms/step - loss: 0.1424 - accuracy: 0.9716 - val_loss: 0.0742 - val_accuracy: 0.9852
Epoch 2/10
59/59 [==============================] - 13s 220ms/step - loss: 0.1460 - accuracy: 0.9677 - val_loss: 0.0518 - val_accuracy: 0.9871
Epoch 3/10
59/59 [==============================] - 13s 219ms/step - loss: 0.0956 - accuracy: 0.9776 - val_loss: 0.0459 - val_accuracy: 0.9880
Epoch 4/10
59/59 [==============================] - 13s 220ms/step - loss: 0.0941 - accuracy: 0.9755 - val_loss: 0.0545 - val_accuracy: 0.9866
Epoch 5/10
59/59 [==============================] - 13s 219ms/step - loss: 0.0993 - accuracy: 0.9783 - val_loss: 0.0565 - val_accuracy: 0.9860
Epoch 6/10
59/59 [==============================] - 13s 220ms/step - loss: 0.0797 - accuracy: 0.9771 - val_loss: 0.0567 - val_accuracy: 0.9861
Epoch 7/10
59/59 [==============================] - 13s 220ms/step - loss: 0.0860 - accuracy: 0.9749 - val_loss: 0.0459 - val_accuracy: 0.9

In [185]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

11
Epoch 1/10
64/64 [==============================] - 15s 213ms/step - loss: 0.2415 - accuracy: 0.9589 - val_loss: 0.0452 - val_accuracy: 0.9870
Epoch 2/10
64/64 [==============================] - 14s 214ms/step - loss: 0.1209 - accuracy: 0.9709 - val_loss: 0.0982 - val_accuracy: 0.9862
Epoch 3/10
64/64 [==============================] - 14s 213ms/step - loss: 0.0963 - accuracy: 0.9725 - val_loss: 0.0499 - val_accuracy: 0.9876
Epoch 4/10
64/64 [==============================] - 14s 213ms/step - loss: 0.1215 - accuracy: 0.9715 - val_loss: 0.0432 - val_accuracy: 0.9881
Epoch 5/10
64/64 [==============================] - 13s 211ms/step - loss: 0.0808 - accuracy: 0.9765 - val_loss: 0.0473 - val_accuracy: 0.9869
Epoch 6/10
64/64 [==============================] - 13s 212ms/step - loss: 0.0760 - accuracy: 0.9792 - val_loss: 0.0494 - val_accuracy: 0.9874
Epoch 7/10
64/64 [==============================] - 14s 213ms/step - loss: 0.0714 - accuracy: 0.9799 - val_loss: 0.0460 - val_accuracy: 0.9

In [186]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

12
Epoch 1/10
70/70 [==============================] - 16s 206ms/step - loss: 0.1686 - accuracy: 0.9596 - val_loss: 0.0814 - val_accuracy: 0.9829
Epoch 2/10
70/70 [==============================] - 14s 203ms/step - loss: 0.1322 - accuracy: 0.9660 - val_loss: 0.1219 - val_accuracy: 0.9767
Epoch 3/10
70/70 [==============================] - 14s 204ms/step - loss: 0.1307 - accuracy: 0.9682 - val_loss: 0.0539 - val_accuracy: 0.9847
Epoch 4/10
70/70 [==============================] - 14s 203ms/step - loss: 0.1022 - accuracy: 0.9743 - val_loss: 0.0780 - val_accuracy: 0.9822
Epoch 5/10
70/70 [==============================] - 14s 208ms/step - loss: 0.0881 - accuracy: 0.9780 - val_loss: 0.0469 - val_accuracy: 0.9867
Epoch 6/10
70/70 [==============================] - 14s 203ms/step - loss: 0.0892 - accuracy: 0.9768 - val_loss: 0.0549 - val_accuracy: 0.9843
Epoch 7/10
70/70 [==============================] - 14s 204ms/step - loss: 0.0767 - accuracy: 0.9801 - val_loss: 0.0464 - val_accuracy: 0.9

In [187]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

13
Epoch 1/10
75/75 [==============================] - 16s 202ms/step - loss: 0.1934 - accuracy: 0.9601 - val_loss: 0.0442 - val_accuracy: 0.9881
Epoch 2/10
75/75 [==============================] - 15s 201ms/step - loss: 0.1134 - accuracy: 0.9737 - val_loss: 0.0403 - val_accuracy: 0.9883
Epoch 3/10
75/75 [==============================] - 15s 201ms/step - loss: 0.1005 - accuracy: 0.9769 - val_loss: 0.0448 - val_accuracy: 0.9874
Epoch 4/10
75/75 [==============================] - 15s 201ms/step - loss: 0.0889 - accuracy: 0.9754 - val_loss: 0.0460 - val_accuracy: 0.9857
Epoch 5/10
75/75 [==============================] - 15s 203ms/step - loss: 0.0852 - accuracy: 0.9716 - val_loss: 0.0456 - val_accuracy: 0.9875
Epoch 6/10
75/75 [==============================] - 15s 201ms/step - loss: 0.0913 - accuracy: 0.9765 - val_loss: 0.0554 - val_accuracy: 0.9873
Epoch 7/10
75/75 [==============================] - 15s 201ms/step - loss: 0.0777 - accuracy: 0.9797 - val_loss: 0.0718 - val_accuracy: 0.9

In [188]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

14
Epoch 1/10
80/80 [==============================] - 17s 197ms/step - loss: 0.1707 - accuracy: 0.9636 - val_loss: 0.0777 - val_accuracy: 0.9819
Epoch 2/10
80/80 [==============================] - 16s 196ms/step - loss: 0.1307 - accuracy: 0.9644 - val_loss: 0.0426 - val_accuracy: 0.9885
Epoch 3/10
80/80 [==============================] - 16s 197ms/step - loss: 0.0895 - accuracy: 0.9720 - val_loss: 0.0439 - val_accuracy: 0.9879
Epoch 4/10
80/80 [==============================] - 16s 197ms/step - loss: 0.1239 - accuracy: 0.9717 - val_loss: 0.0551 - val_accuracy: 0.9877
Epoch 5/10
80/80 [==============================] - 16s 196ms/step - loss: 0.1011 - accuracy: 0.9742 - val_loss: 0.0549 - val_accuracy: 0.9854
Epoch 6/10
80/80 [==============================] - 17s 213ms/step - loss: 0.0776 - accuracy: 0.9785 - val_loss: 0.0522 - val_accuracy: 0.9850
Epoch 7/10
80/80 [==============================] - 17s 219ms/step - loss: 0.0983 - accuracy: 0.9764 - val_loss: 0.0526 - val_accuracy: 0.9

In [189]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

15
Epoch 1/10
85/85 [==============================] - 20s 218ms/step - loss: 0.2011 - accuracy: 0.9555 - val_loss: 0.0471 - val_accuracy: 0.9864
Epoch 2/10
85/85 [==============================] - 18s 216ms/step - loss: 0.1224 - accuracy: 0.9682 - val_loss: 0.0599 - val_accuracy: 0.9840
Epoch 3/10
85/85 [==============================] - 18s 216ms/step - loss: 0.1157 - accuracy: 0.9715 - val_loss: 0.0450 - val_accuracy: 0.9884
Epoch 4/10
85/85 [==============================] - 18s 216ms/step - loss: 0.1004 - accuracy: 0.9750 - val_loss: 0.0525 - val_accuracy: 0.9871
Epoch 5/10
85/85 [==============================] - 18s 216ms/step - loss: 0.0996 - accuracy: 0.9731 - val_loss: 0.0473 - val_accuracy: 0.9870
Epoch 6/10
85/85 [==============================] - 18s 217ms/step - loss: 0.0919 - accuracy: 0.9751 - val_loss: 0.0461 - val_accuracy: 0.9881
Epoch 7/10
85/85 [==============================] - 18s 215ms/step - loss: 0.1058 - accuracy: 0.9760 - val_loss: 0.0579 - val_accuracy: 0.9

In [190]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

16
Epoch 1/10
91/91 [==============================] - 22s 216ms/step - loss: 0.1467 - accuracy: 0.9642 - val_loss: 0.0451 - val_accuracy: 0.9881
Epoch 2/10
91/91 [==============================] - 20s 216ms/step - loss: 0.1263 - accuracy: 0.9666 - val_loss: 0.0621 - val_accuracy: 0.9855
Epoch 3/10
91/91 [==============================] - 20s 215ms/step - loss: 0.1310 - accuracy: 0.9687 - val_loss: 0.0633 - val_accuracy: 0.9860
Epoch 4/10
91/91 [==============================] - 19s 214ms/step - loss: 0.1051 - accuracy: 0.9712 - val_loss: 0.0466 - val_accuracy: 0.9872
Epoch 5/10
91/91 [==============================] - 19s 215ms/step - loss: 0.1058 - accuracy: 0.9731 - val_loss: 0.0403 - val_accuracy: 0.9886
Epoch 6/10
91/91 [==============================] - 19s 215ms/step - loss: 0.0995 - accuracy: 0.9718 - val_loss: 0.1025 - val_accuracy: 0.9836
Epoch 7/10
91/91 [==============================] - 19s 214ms/step - loss: 0.0895 - accuracy: 0.9789 - val_loss: 0.0552 - val_accuracy: 0.9

In [191]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

17
Epoch 1/10
96/96 [==============================] - 22s 212ms/step - loss: 0.1736 - accuracy: 0.9627 - val_loss: 0.0449 - val_accuracy: 0.9877
Epoch 2/10
96/96 [==============================] - 21s 214ms/step - loss: 0.1290 - accuracy: 0.9690 - val_loss: 0.0387 - val_accuracy: 0.9890
Epoch 3/10
96/96 [==============================] - 22s 225ms/step - loss: 0.1095 - accuracy: 0.9731 - val_loss: 0.0559 - val_accuracy: 0.9856
Epoch 4/10
96/96 [==============================] - 21s 218ms/step - loss: 0.1178 - accuracy: 0.9692 - val_loss: 0.0626 - val_accuracy: 0.9845
Epoch 5/10
96/96 [==============================] - 20s 213ms/step - loss: 0.1039 - accuracy: 0.9722 - val_loss: 0.0530 - val_accuracy: 0.9847
Epoch 6/10
96/96 [==============================] - 20s 214ms/step - loss: 0.1040 - accuracy: 0.9736 - val_loss: 0.0470 - val_accuracy: 0.9870
Epoch 7/10
96/96 [==============================] - 21s 216ms/step - loss: 0.0894 - accuracy: 0.9785 - val_loss: 0.0562 - val_accuracy: 0.9

In [192]:
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

18
Epoch 1/10
101/101 [==============================] - 23s 212ms/step - loss: 0.1670 - accuracy: 0.9604 - val_loss: 0.0502 - val_accuracy: 0.9862
Epoch 2/10
101/101 [==============================] - 21s 212ms/step - loss: 0.1262 - accuracy: 0.9686 - val_loss: 0.0574 - val_accuracy: 0.9863
Epoch 3/10
101/101 [==============================] - 21s 211ms/step - loss: 0.1110 - accuracy: 0.9717 - val_loss: 0.0652 - val_accuracy: 0.9855
Epoch 4/10
101/101 [==============================] - 21s 210ms/step - loss: 0.0984 - accuracy: 0.9763 - val_loss: 0.0489 - val_accuracy: 0.9876
Epoch 5/10
101/101 [==============================] - 21s 211ms/step - loss: 0.0936 - accuracy: 0.9750 - val_loss: 0.0536 - val_accuracy: 0.9864
Epoch 6/10
101/101 [==============================] - 21s 212ms/step - loss: 0.1094 - accuracy: 0.9781 - val_loss: 0.0505 - val_accuracy: 0.9849
Epoch 7/10
101/101 [==============================] - 21s 212ms/step - loss: 0.0808 - accuracy: 0.9746 - val_loss: 0.0458 - val

In [193]:
#
print(n)
models_nc[n].fit_model(image_sets_nc[n],label_sets_nc[n],x_val,y_val)
n = n+1

19
Epoch 1/10
110/110 [==============================] - 25s 208ms/step - loss: 0.1882 - accuracy: 0.9582 - val_loss: 0.0619 - val_accuracy: 0.9859
Epoch 2/10
110/110 [==============================] - 23s 207ms/step - loss: 0.1356 - accuracy: 0.9654 - val_loss: 0.0428 - val_accuracy: 0.9877
Epoch 3/10
110/110 [==============================] - 23s 207ms/step - loss: 0.1158 - accuracy: 0.9719 - val_loss: 0.0603 - val_accuracy: 0.9848
Epoch 4/10
110/110 [==============================] - 23s 207ms/step - loss: 0.1185 - accuracy: 0.9712 - val_loss: 0.0454 - val_accuracy: 0.9870
Epoch 5/10
110/110 [==============================] - 23s 206ms/step - loss: 0.1142 - accuracy: 0.9723 - val_loss: 0.0594 - val_accuracy: 0.9867
Epoch 6/10
110/110 [==============================] - 23s 206ms/step - loss: 0.0815 - accuracy: 0.9789 - val_loss: 0.0524 - val_accuracy: 0.9854
Epoch 7/10
110/110 [==============================] - 23s 207ms/step - loss: 0.0831 - accuracy: 0.9800 - val_loss: 0.0546 - val

In [194]:
#new_model_nc_dir  = "C:/Users/fjdurlop/Documents/upc/models/"+dataset+"/C3/"+dataset+"_model_c3_aug_nc_e1"
new_model_nc_dir  = "D:/models/aug_22/"+dataset+"/C3/"+dataset+"_model_c3_aug_nc_e1"
i=0

for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1

INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_0\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_1\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_2\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_3\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_4\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_5\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_6\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model_c3_aug_nc_e1_7\assets
Model has been saved
INFO:tensorflow:Assets written to: D:/models/aug_22/mnist/C3/mnist_model

In [ ]:
try:
    del nc_values
    del top_images_by_nc
    del top_labels_by_nc
    del image_sets_nc
    del label_sets_nc
    del models_nc
except:
    print("Error")

In [ ]:
gc.collect()

## Evaluating

In [ ]:
model_original.model.compile(loss= 'categorical_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy',tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [ ]:
# Metrics using adversarial test
evaluate_lsa_0 = []
evaluate_dsa_0 = []
evaluate_nc_0 = []
evaluate_random_0 = []

evaluate_lsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_dsa_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_nc_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
evaluate_random_0.append(model_original.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

for model in models_lsa:
    evaluate_lsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_dsa:
    evaluate_dsa_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_random:
    evaluate_random_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))
    
for model in models_nc:
    evaluate_nc_0.append(model.evaluate(x_adversary_test_fgsm,y_adversary_test_fgsm))

In [ ]:
evaluate_lsa_0[0]=[0,0,0,0]
evaluate_dsa_0[0]=[0,0,0,0]
evaluate_random_0[0]=[0,0,0,0]
evaluate_nc_0[0]=[0,0,0,0]

In [ ]:
print(len(evaluate_lsa_0))
print(len(evaluate_dsa_0))
print(len(evaluate_random_0))
print(len(evaluate_nc_0))

In [ ]:
# Metrics using original test
evaluate_lsa_1 = []
evaluate_dsa_1 = []
evaluate_nc_1 = []
evaluate_random_1 = []

evaluate_lsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_dsa_1.append(model_original.evaluate(x_test,y_test))
evaluate_nc_1.append(model_original.evaluate(x_test,y_test))
evaluate_random_1.append(model_original.evaluate(x_test,y_test))
print("lsa---")
for model in models_lsa:
    evaluate_lsa_1.append(model.evaluate(x_test,y_test))

print("dsa---")
    
for model in models_dsa:
    evaluate_dsa_1.append(model.evaluate(x_test,y_test))
    

print("random---")

for model in models_random:
    evaluate_random_1.append(model.evaluate(x_test,y_test))
    
print("nc---")

for model in models_nc:
    evaluate_nc_1.append(model.evaluate(x_test,y_test))


In [ ]:
evaluate_lsa_1[0]=[0,0,0,0]
evaluate_dsa_1[0]=[0,0,0,0]
evaluate_random_1[0]=[0,0,0,0]
evaluate_nc_1[0]=[0,0,0,0]

In [ ]:

print(len(evaluate_lsa_1))
print(len(evaluate_dsa_1))
print(len(evaluate_random_1))
print(len(evaluate_nc_1))

In [ ]:
type(evaluate_dsa_0[0][0])

In [ ]:
import pandas as pd

df_evaluate_lsa_0 = pd.DataFrame(np.array(evaluate_lsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_lsa_1 = pd.DataFrame(np.array(evaluate_lsa_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_0 = pd.DataFrame(np.array(evaluate_dsa_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_dsa_1 = pd.DataFrame(np.array(evaluate_dsa_1),columns=["loss","accuracy","precision","recall"])

df_evaluate_random_0 = pd.DataFrame(np.array(evaluate_random_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_random_1 = pd.DataFrame(np.array(evaluate_random_1),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_0 = pd.DataFrame(np.array(evaluate_nc_0),columns=["loss","accuracy","precision","recall"])
df_evaluate_nc_1 = pd.DataFrame(np.array(evaluate_nc_1),columns=["loss","accuracy","precision","recall"])


In [ ]:
# Original test set + adversarial set

metric ="accuracy"
accuracy_lsa_3 = (np.array(df_evaluate_lsa_0[metric])+np.array(df_evaluate_lsa_1[metric]))/2
accuracy_dsa_3 = (np.array(df_evaluate_dsa_0[metric])+np.array(df_evaluate_dsa_1[metric]))/2
accuracy_nc_3 = (np.array(df_evaluate_nc_0[metric])+np.array(df_evaluate_nc_1[metric]))/2
accuracy_random_3 = (np.array(df_evaluate_random_0[metric])+np.array(df_evaluate_random_1[metric]))/2


## Charts

In [ ]:
n_inputs = [1800*i for i in range(20)]
n_inputs.append(len(x_train_and_adversary))

In [ ]:
print(n_inputs)

In [ ]:
cd "C:/Users/fjdur/Desktop/gtsrb_graphs/e2_graphs_dec/"

In [ ]:
linestyles = ['solid','dotted','dashed','dashdot']
colors =['k','k','k','k']

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

#metric = "accuracy" # accuracy loss
my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_0[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_0[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_0[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_0[metric],colors[3],linestyle=linestyles[3])


    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with test set of adversarial examples FGSM")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_0.svg")
    plt.show()

In [ ]:
#configuration 1
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

my_metrics =["accuracy","loss","precision","recall"]

for metric in my_metrics:
    
#metric = "accuracy" # accuracy loss
    plt.clf()
    plt.plot(n_inputs,df_evaluate_lsa_1[metric],colors[0],linestyle=linestyles[0])
    plt.plot(n_inputs,df_evaluate_dsa_1[metric],colors[1],linestyle=linestyles[1])
    plt.plot(n_inputs,df_evaluate_random_1[metric],colors[2],linestyle=linestyles[2])
    plt.plot(n_inputs,df_evaluate_nc_1[metric],colors[3],linestyle=linestyles[3])

    legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



    plt.legend(handles=legend_elements)#
    plt.title(metric + " with original set")
    plt.xlim([0, 37000])
    #plt.ylim([0, 1])
    plt.xlabel('number of inputs')
    plt.ylabel(metric)
    plt.savefig("gtsrb_c1_"+metric + "_1.svg")
    plt.show()

In [ ]:
#adversarial jsma test set
#configuration 3
"""
6) Incremental guided retraining starting from the original model using only the new adversarial inputs.
Incremental training, starting with the previous trained model. Using at each iteration a subset of the new inputs.
"""
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.plot(n_inputs,accuracy_lsa_3,colors[0],linestyle=linestyles[0])
plt.plot(n_inputs,accuracy_dsa_3,colors[1],linestyle=linestyles[1])
plt.plot(n_inputs,accuracy_random_3,colors[2],linestyle=linestyles[2])
plt.plot(n_inputs,accuracy_nc_3,colors[3],linestyle=linestyles[3])


legend_elements = [Line2D([0], [0], color='k', label='LSA',ls = linestyles[0]),
                   Line2D([0], [0], color='k', label='DSA',ls = linestyles[1]),
                   Line2D([0], [0], color='k', label='Random',ls = linestyles[2]),
                   Line2D([0], [0], color='k', label='NC',ls = linestyles[3])]



plt.legend(handles=legend_elements)#
plt.title("accuracy with both sets")

print(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max())
print(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max())
print(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max())
print(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max())

plt.plot(n_inputs[np.argmax(accuracy_lsa_3)],accuracy_lsa_3.max(),'-kD')
plt.plot(n_inputs[np.argmax(accuracy_dsa_3)],accuracy_dsa_3.max(),'-ko')
plt.plot(n_inputs[np.argmax(accuracy_random_3)],accuracy_random_3.max(),'-kv')
plt.plot(n_inputs[np.argmax(accuracy_nc_3)],accuracy_nc_3.max(),'-kp')

plt.xlabel('number of inputs')
plt.ylabel('accuracy')
plt.xlim([0, 37000])
#plt.ylim([0, 1])

plt.savefig("gtsrb_c1_"+"accuracy" + "_both.svg")

plt.show()

## Saving models

In [ ]:
new_model_lsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_lsa_e2"
i=0

for model in models_lsa:
    model.save(new_model_lsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_dsa_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_dsa_e2"

i=0
for model in models_dsa:
    model.save(new_model_dsa_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_random_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_random_e2"

i=0
for model in models_random:
    model.save(new_model_random_dir+"_"+str(i))
    i+=1

In [ ]:
new_model_nc_dir  = "D:/models/gtsrb_models/C1/gtsrb_model_c1_sep_nc_e2"

i=0
for model in models_nc:
    model.save(new_model_nc_dir+"_"+str(i))
    i+=1